<a href="https://colab.research.google.com/github/TejaswiniVB/TensorflowTfRec/blob/main/Tfrecord_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)

2.15.0


In [ ]:
import os
os.path.abspath("")

'/content'

In [ ]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython']

In [ ]:
testDataset = tf.data.TFRecordDataset("./Shapes/test/Shape.tfrecord")

In [ ]:
pip install tf-models-official # Done

In [ ]:
from functools import partial
import matplotlib.pyplot as plt

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
GCS_PATH = "gs://kds-b38ce1b823c3ae623f5691483dbaa0f0363f04b0d6a90b63cf69946e"
BATCH_SIZE = 64
IMAGE_SIZE = [1024, 1024]

In [ ]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

In [ ]:
def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
            "image": tf.io.FixedLenFeature([], tf.string, default_value=''),
            "target": tf.io.FixedLenFeature([], tf.int64, default_value=0),
        }
        if labeled
        else {"image": tf.io.FixedLenFeature([], tf.string),}
    )
    print(tfrecord_format)
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    if labeled:
        label = tf.cast(example["target"], tf.int32)
        return image, label
    return image

In [ ]:
def load_dataset(filenames, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

In [ ]:
def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames, labeled=labeled)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [ ]:
TRAINING_FILENAMES = tf.io.gfile.glob("./Shapes/train/Shape.tfrecord")
TEST_FILENAMES = tf.io.gfile.glob("./Shapes/test/Shape.tfrecord")
VALID_FILENAMES = tf.io.gfile.glob("./Shapes/valid/Shape.tfrecord")

In [ ]:
train_dataset = get_dataset(TRAINING_FILENAMES)
valid_dataset = get_dataset(VALID_FILENAMES)
test_dataset = get_dataset(TEST_FILENAMES, labeled=False)

{'image': FixedLenFeature(shape=[], dtype=tf.string, default_value=''), 'target': FixedLenFeature(shape=[], dtype=tf.int64, default_value=0)}
{'image': FixedLenFeature(shape=[], dtype=tf.string, default_value=''), 'target': FixedLenFeature(shape=[], dtype=tf.int64, default_value=0)}
{'image': FixedLenFeature(shape=[], dtype=tf.string, default_value=None)}


In [ ]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 1024, 1024, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
train_ds = train_dataset.unbatch()

In [ ]:
for str_rec in tf.python_io.tf_record_iterator('./Shapes/train/Shape.tfrecord'):
    example = tf.train.Example()
    example.ParseFromString(str_rec)
    print(dict(example.features.feature).keys())

In [ ]:
for str_rec in tf.data.TFRecordDataset('./Shapes/train/Shape.tfrecord'):
    output = repr(str_rec)
    print(<tf.Tensor: shape=(), dtype=string, numpy=b'\n\x84\xfe\x01\n\xf6\xfa\x01\n\rimage/encoded\x12\xe3\xfa\x01\n\xdf\xfa\x01\n\xdb\xfa\x01\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x018\x019\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xe1\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xf1\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xff\xc4\x00\x1f\x01\x00\x03\x01\x01\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x11\x00\x02\x01\x02\x04\x04\x03\x04\x07\x05\x04\x04\x00\x01\x02w\x00\x01\x02\x03\x11\x04\x05!1\x06\x12AQ\x07aq\x13"2\x81\x08\x14B\x91\xa1\xb1\xc1\t#3R\xf0\x15br\xd1\n\x16$4\xe1%\xf1\x17\x18\x19\x1a&\'()*56789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xff\xda\x00\x0c\x03\x01\x00\x02\x11\x03\x11\x00?\x00\xf6\x9d\x1e\xe2\x1b\x7f\r\xe9o<\xd1\xc4\x86\xde\x04\x0c\xec\x14\x16`\xaa\xa3\x9e\xe5\x88\x00w$\n\xd5\x15\xc8\xfc:\xba\x9a\xeb\xc1\x1ad\xb3\xb2\xb3\x18\x86\n\xbe\xe0W\xf8Np;c\xe9\xeaz\xd7Z\r\x008\xd5k{+{id\x96(\xff\x00}\'\x0f+\xb1wa\xb9\x98)c\x93\xb4\x17|.p\xb9\xc0\x00qR4\xf1\x8e\xac\xbf\x9dU\x92\xe2\xc0]\xa4\xd2\x18~\xd1\x1a2$\x84\r\xca\xacA`\x0fP\tU\xc8\xef\xb4zP\x05\xa9\xa0\x86\xe5\x02O\x14r\xa0up\xae\xa1\x80e`\xcay\xee\x18\x02\x0fb\x01\xa1\xa1V\xb8I\xc9\x93z# \x02F\x0b\x86 \x9c\xaepO\xca0H\xc8\xe7\x18\xc9\xcdY\xb5kH\x08\x0f(\x04\x8c\xfe\x15V\x7f\x11X\xc3\x1e\xe3(\xa0\r\x8c\xd4q\xb4\xc5\xe6\x12\xa4j\x81\xf1\x11W,Yv\x8eX`m;\xb7\x0c\x0c\xf0\x01\xcf8\x1c\xcd\xd7\x8a\xf4{\x88\xa4\xb2\xb81L\xb3\xc4\xc1\xa0\x93\x0c$N\x03\x02\xa7\xa8\xf9\x80=\xb9\xf7\xa8S\xc6qM\x13I\x14/\x84\xc8;\xc6\xd2p\xdbx\x07\x04\xfe\x19\xc8\xe4q\xcd\x00tm4+%\xed\xc5\xb1{\xab\x88PG%\xbcS\x82w*\x97T\n\xcc\x11\x1c\x87\x1c\x9d\xb9\x05rp\x06.f\xbc\xfe\xeb\xc5\xd7{\xe0e\x1e^\x18\xb9Ut!\xd7\x05B\xb0 \x91\xcf<\x10x\x1d\xb3\x9c\xeb\xdf\x1b\xdd#\x82\'\x8d\x0eTya\x86@$\x02y#\x81\xd4\xf7\xc0\xe3\'\x8a\x00\xf4O,\xdc\xc9c|\xedui"!-ld\\\x10\xcb\xcaH\x01e$\x10\x0e\xe59\x05p\x1bk0e\x8e\xf9\xcc\x924\xf0-\xbc#\x84/(.\xc43\x02J\x8c\x80\xa4\x04`wd\xee\xc1\nG>]\xa9x\xd8\xdb k\x8dE[\n\x1b\xfd\x1f\xe78#=\xbb\xfbu\xed\xd6\xb3f\xf1\x92\\\xa9U\xb8\x9aR\x01l3\x08\xc6\x00$\xf2O^8\x1dI\xe0rh\x03\xd9\x1bS\xb4E$\xcc\xbc{\xd5$\xf1>\x9a\xf1y\x82|\r\xcc\xb8e*xb\xa7\x83\xdb#\x83\xdc`\x8c\x82\ry\x05\xb6\xb9f!;\x02\xc2\x84\xb3\x95\xdf\x8c\x92I\'\x03\xb9$\x93\xeaMS\x9b\xc4\xb6\x96\xd3Kp\xb0%\xc1o\x94\x88\xa3\xd9&\xd0\xa7h\xc9\xce\xec9\'\x9d\xa3\x0cy\x18\xc9\x00\xf5\xab\xcf\x1cY\xc1\x91\x02\t\x98:\x8eX\xa8+\xb8n \xe0\xf2\x06H\x18\xe4\xe0dg"\xa4\xde<r\xd9\xb5\xd3\xa6\x92?\xefc\x15\xe5m\xe2\xc9!,\xc8\xaf\xb1F~E\xcf\xe4:\xd4M\xe2\x9b\xc9\x18\x97\x8f\x07\'\x1f\xbc\xed\x9e=;\x7f\x93@\x1e\x98\x9e>\xb9\xbf_\xf48!\x18\x8df%\xa6^Q\x8e\x15\x87<\x83\x83\x8cu\xc1\xf44\xd9|]t\xe0C$\xd6\xf1\xc8\xc0\xb8]\xc0\xb1\x03\x00\x90:\xe0d~b\xbc\xd8\xebWs\xde\xcb\x02\xe3t\x08\xea\xebo\x89\xc6\xe4\x0c\xdb\xb8=1\x8c\x90p\x15C\x00y\xcc0k7\xf7,\xd1\xeea\xfb\xb7\x7f\xdc\xc3\xb9\x86\xd5-\xd3#\x8e9=\x86O8\xc5\x00wZ\x86\xbf-\xcc\x1eD\xb24\xb0\xb8+$>He\x99J\x90Q\x81\x07\x8es\xdb\xa0\xed\x90j\xff\x00lj(\xce\x97:|\xe5Ko\x8c\x04\nB\x95\x1dI<\xfa\x8e\x83\x04q\xd4\x9e\r\xb5+\x88\xaen!\x8a\xf89-\xb9\xde.<\xcf\xe1\r\xc8\x07\x90\xb8\x19\x00\xe0{Uw\x96\xefi\x9b\xcf"\x15uS\xfb\xc2]\x8bn\xe8\xbdp02\xdc\x81\xdf\x19\x06\x80;\xb8\xf5\xbb\xdbDX\xe0\x8e\xdd\x15\x00P..\x0b>\x07\xf7\x89\xc9\'\xdc\x9c\xd4w\x1a\xde\xa7=\xef\x95.\xafc\x10-\xb2-\x87*Fz\xe5\xb1\xb4s\x9eq\xef\\;\xc5\xe4C\x11r\\8\x1bN\xff\x001\xba\xed\xf9\xb0I\x07#\xbf8\xe7\xa1\xcd9\x94#\x15\r\x1b\xe0\xe3tn\x1dO\xb8`H#\xdc\x1c\x1a\x00\xd9\x96\xe1f&\xee\xe6\xf2c\x03\xce\xe1L\x11\x94\x13:\xaa\xa9/\x902@\xc0\xc6H\xc6\xd3\xfd\xdcV7:L?\xbcHnn\x1f#\xe5f\x18\xeb\xfeMg$\xb0\xc5<~tr<e\x86\xe5\x8c\x10H\xc8\x04n\xc1\x00\xf3\xd4\x83\xeb\x8c\x03S-\xbb\xb3\xc0\x80\xc4\x0c\xd8\xdb\x99T\x01\xc9_\x98\x93\x85\xe4\x7f\x168\xe7\xa1\xcd\x00i\xa6\xbd\x04{\x994\x88\xf7\xaf\xdc\xdc\xf8\x0cq\xc7#8\x19\xe3\xa7n\x95d\xf8\xbe\xf01\xdb\x04@v\xaeq]\\eX0\xc9\x19\x07<\x83\x83\xfa\xd4\x8f,\x92,j\xee\xcc\xb1\xae\xd4\x04\xe4(\xc98\x1e\x83$\x9f\xa94\x01\xa8\xde\'\xd4\'%\xc4\xa59#\x1bq\x8c\x1cw\xaawZ\xad\xd5\xdb*K<\x9b\xbe\xf0+\x91\xd0\x8e\xe3\xf0\xe3\xbf=Fj\xaeicH|\xd6i2\x84\xa1\x1b\x95\x01$\x80J\x83\xc8\xe3\'\xf0\xc98=\x08\x05\x89.K\xdb\xf9o\xa88\xde\x0342d\t$\x04\x80\x17\x19\xdcB\x92\xd98\xfe!\xf5\xaf+\xbc\xcd\xbaY\x1eB\x14(.\xd9\xc0\x00\x00>\x80\x00\x07\xb0\xa5\x8e6\x95\x8a\xa9PB\x96\xf9\x98(\xc0\x04\x9eO~8\x1dI\xe0sL\x95\xa3X\x15\x80\x94:33\x95;\x83\xae8\x01q\x9c\xe7\'\x83\xcf\x1e\x87 \x0b%\xb9\x85\x82\xcb\x13#m\x0c\x03\x0c\x1c\x10\x08?B\x08#\xd8\xd4o"\xc5\xb7!\x8e\xe6\n6\xa9n}\xf1\xd0{\x9e*c8\x8d\xe4\x8a\xdeY\xb6<j$\xca\x94\r\x9c\x12\xa7\x04\x82\x03/\xe8\x0e\x07J\x89\x99Pe\x98(\xc8\x19\'\x1c\x93\x81\xfa\xd0\x02\xe6\x9aTIsn\x8d<\xb0\xc5\xe6\x06\x95\xa3\x889(:\x8ep3\xc88\xc8\xce:\x8e\xa2Yn\xe0\x92\xd9$X\xa0\x8a5]\xc6Tf\xc3\r\xa3\x92K\x11\x8e\x0bq\x8eX\xf6\xc0\x17\x0f\xdag\xba\x94\xc3h\x97\xe9o#\\H-\xe0\xda\x85271\xda\x15\x96<\x01\xc7\x1bA8\x00\x9a\x00\xa1\x02\t\x84\xb2[\xc6]B,\xd2H\x88J\xed$(b\xc3\x8f\xee\x8f\xc4z\x8a\x928\xdaV*\xa5A\n[\xe6`\xa3\x00\x12y=\xf8\xe0u\'\x81\xcd:\xfa\xe6\xe2\xf2\xeeY\xe4\xb9\x96y\x1aN&\x9c\x96w@p\x0bs\xc1*\x07\x198\xf5 s\tEb\xa5\x94\x12\xa7*H\xe8q\x8e?\x02h\x02Hd\x92\t\x92h\x9d\xa3\x91\x182:\x1c\x15#\x90A\xeci\xd6A\x1fV\xb6Y\x03\xacm \xdd"\x80v\x80Gl\x8c\x9f\xf0\xea*\xbbH\xc8\xe0ye\x95\x88\x0b\xb7\x93\x9es\x9e\xc0\x01\xces\xed\xd7\x19\xb1\x1b\xf9s\xc6\xd8\xce\xd6\r@\x1e\xdf\xe1\x98d\x92\xf1\xe4Y]\x122\xa5\xc2\xed\xc4\x99R\x02\x9c\x82q\xdf\x8c\x1c\xa8\xe7\x19\x07\xad\xc1\xf5\xaeS\xc3\xdf%\xe2\xbcYx&\x8c\x10\xe5q\x9e\x07\x1f^FG?\x8du\x994\x01\xc7\xfc<G\xff\x00\x84GH\x90\xdc6\x16\xdbkD\xaa\x02\xb1;~c\x91\x9c\x8c\x1c`\xe3\x0cs\x93\x8ct\xf7\x1fg\x92\xea\x08\xe5\x87|\xc8\x1ah\\\xc2XF@\xd8Hla[\x12\x11\xd4\x12\x0bc \x1a\xf3\xaf\x83\x1a\x8c\xf7\xde\x07\xb77\x12\x17tr\x83\x8e\x02\x8e\x95\xe9\x89\x1by\xef)\x99\xca2*\x88\x88]\xaaA9a\xc6rr\x01\xc9#\xe5\x18\x03\x9c\x80y/\x89u=GO\xd5&\x02P\xb1\x10\x08\x0c\xa7 \xf2s\x9c\xf2\x08#\x1cz\xf5\xcf\x1c\x9f\xfc%3\xdc\xe2d\xbc\x91\x95\xba\x14^\x0f\xb8\xcfo~\xf5\xda\xf8\xf3o\xdb\xe5\x8b\xc9*\xc1w\xef(\xc009\x00\x03\xd0\xe0\xa9\xe0r3\xcfQ^]\xf4\xf54\x01\xac5\x8b\xdb\xc6\\\xbd\xc4\x93\x04,\xc9\x1b\x12\x14\x01\x96<\x0e@\x00\x9c\xe0p3QE\xae\xb34G\x0f,\x0cC6%\xfb\xcb\xecp@\xc8\xef\x83\xf8\xd5dH\xd6\xe6I#\x0b<\x10\xb6\xfd\xb3\x9f,\xc8\x9b\x80\x00\x80\xd9\xc9\xc8\xc8RH\x19\xe7\x8c\xd4\x11\x88\x84\xbb\xe5Y\x18\x04`\x02>\xdeH\xe0\x9e\x0888=:d\x023\x9a\x00\xb9\xfd\xaf{\x82\x04\x8a>\x82\xa3k\x9b\xcbi\x1b\x17%ZM\xb21\x8d\x94\xee%F\t#\xbe0=F0zb\xabQ@\x12\x06\x9e\xe6RL\xc4\xb6\xd6b^@\xa3\x80I\xeaz\xf1\xc0\xeaO\x03\x93QD\xaa\xd7+\xe7\xca\xc2\x12@b\xa9\xb9\x94g\x922@<t\x1ct\xeb\xcf\x12\xe5\xa7i\xe6\x9aHVBK\x85X\xf6\x06\xcb}\xd5U\x18\x18\x07=\x87\x07\x18\xe0S\x12\xdd\x96\xdc\xbcp\x91\x04eSr\xaf\xca\xa4\x83\xb4z\x0e\x14\xe0{\x1fJ\x00\x8e\tJ\xb3\x90".7#\x02\x03\x81\x91\x8e\x84ppA\xf5\x04\x82\x0f\x00\xd4\x8260\xbc\xa0\xae\xd0\xc1H.3\x93\x9e\x83\xa9\x1c\x1eG\x03\x8c\xf5\x14\xe8J\xa4\xc8\xed\x12\xca\xaa\xc0\x98\xdc\x90\x18z\x1c\x10p}\x884\xd8\xc1\xb6\x95\xe5\x8e0\xc6A\xb5\x94\xa2\x11\xc8\xda[\x91\xd7\x07\xafQ\xb4c\x91@\x02\xdb\xa2\xcc\xcb\xf6\xa8\xca\x94\xde\x18\x86\xda\x0e\xcc\xec\x1f.s\x9e=2z\xed\xe6\xac\xc5y\x13\\F\x868Q\xe5\x87\xc9y\xa5\x8frG\xc8]\xe0*\xe0\x1d\xb8\x1c\x86bw0\xf9\xb0Dx\xb7\x9a\xe6a\x1by\x11)b\xbeq,G\x19\x0b\x90\xbf1<\x0c\x80\x06z\xe3\x9c\ro\x1a\xef\xff\x00J\x8c\xedEa\x80\xdf18\xca\x8e:\x8c\x9c\xe7\x03\xe58\'\x8c\x80B\x88\x82\xdc\xee\x92F\x98\x15\x03*0\xc3\x07$\x9e0s\x8e\x00\xe7\'\xa69Yc\x11\xca\xe8\xb2,\xaa\xac@\x91\x01\xc3\x0fQ\x90\x0e\x0f\xb8\x06\x90\xb2\x8e9?Jh\x99\x18\xb0_\x98\xa9\xc3s\xd0\xe38\xfc\x88\xa0\x04\x86H\xe0\x8c\xb1\x88\xc8\x88\x8c\xbb\\9=\x08\xcf\x1c\x92:\x8e\xb9\xc0\xea\x0f/\x95m\xe5y\xc7\xfa\xbbm\xae\xc1e\xcc\x87h\x04\x85;W\xe6\'\xa7@\t=\x87H\xa4r\xa3p\xe1W%\x80R\xc4\x8cv\xc7~\x9e\xb5)\x11\x88\x94\x86\x7f3q\xdc\xa50\xa0q\x82\x0ey=x\xc7\x18\x1c\x9c\xf0\x00\xcb\x98e\x89\xd3i\x8c\xc9\x85?\xeb7.\xd6\xdaO\xdd8\xdd\x8f\xc4\x1e\x0e9\x14\xa3v\xf2r6\xe0`c\x9c\xf7\xe7\xf2\xa4Xd\xf2\xd5b\x07\x03\x03-\x96\xe3\xbf9\xeb\x8e\xff\x00\xce\xac\xb5\x94\xdbah\xdd\x180\xdc\xeb\x83\x95\xe4\x8d\xa78\xe7\x80x\xc8\xc1\xf5\xe8\x01\r*\xa3\x02\xc7q99\x00\xe3\x8e:\x0f\xf3\xde\xac\xae\x9dr\xfbY@\x03?7\xcaNF;zs\x8a\xb9\x1d\x95\xe2\xf9s[\xc5"Mn7+@J\xbb0$\x83\x92p\x1b\xa0\x18\xc0\xe0w\xc9\xa0\x0c\xc6+\xf2\xe0\x11\x84P\xd99\xcb`n#\xd0\x13\x92\x078\x18\x198\xc9\x8b\x04m\x0e\xe9\x96b\x17\xb6z\x9cc<\x9c\x0f\xd0\x9a\xd8\x87\xc3\xd3\\\\<H\xae\xf2F\xa1\xdd\x01\xe5T\x9c\x02}\x01 \x8c\xd5\xc1\xe1;\xb0\xc0\x0bv\xfc\xcd\x00s\x8c\xe9\x19UgPX\xe0s\xd4\xf5\xfe\x86\xa7\x92@\x15!Y"\x91W\xe7\xdc\x89\x83\x96Q\x90I\x00\x9cc\x18\xe8\x0eq\xd7\'\xa8\x97\xc0w\xec\xa5\x1e\xd1]\x19y\x18\xc8 \x8e\xf5\xa9\x07\xc3\xab\xb6\x803\xc6\xea{\x801\x8a\x00\xe0\xe40:\xb4\xb0\x12\xa8duX\x9f%\xd1A\xe0\x93\xb4\x03\x9c\xe3#\xae\t\xc0\x04e\xd6l\xd6\xc6iaT\x01b}\xc3\xcc\xd9\x90\xe0\xa9#\x04\x16?>p3\x9er\x08\xcdz<\x1f\x0c\xa3\xb8\xb5d\x9a9\x15\xd86X\xc9\x8e1\xc0\x18\xe4\x1e\xbc\xe7\xb8\xe9\xd6\xa4\xb4\xf8[\xb2\xe5\xdeI0\x85@\np@\xeb\xcf\xaf9\xfd\x07\xbd\x00yg\x9f$*^\t$I6\x91\xfb\xa6\x01\x98\x11\x82\xbc\xf6`J\x9fbj\xc4\xc1$\xf3\xa5\x89@.\x82Hc\x8d\x1c f\xc1\xd8K\xfc\xca\x00$g\x0cr\xbd\xf3\x9a\xf5\xcb\x1f\x87:h\xb9r\xc0\x93\x0b\xedl\xc4\xca\t\xda\x1b\xe5$a\x86\x18r23\x91\xd4\x106\x1f\xc2z}\x9c\xb1\xb1\xb5\x0fn\xdf#2\x86gWfUL"\xa9\xca\xfc\xc4\x96$\x05\xda\t\xc8\xc9\x00\x1e\x13=\x94\xee\x90\x88\xed\xe5R9\x90\x92N\xe3\x92F1\x8c\x0f\xbb\xc1\xcep}p\x1d\x1e\x9d\xa9>\xdc\xd9\xb8\xdc\xe5T\x80X\x11\x8c\x83\x91\xd0c\xd7\x1c\xf1\xcf\x19\xfa\x12\x7f\x0cY\xcb\n,Q\xc2\x8c\xac\xbc\xbc{\x81]\xc0\xb0\xc0#\x922\x01\xcf\x04\x83\x82\x06\r\xd8\xf4k\x18\xa3\x08\xb6\xf1\xe0{P\x07\x80\x0f\t\xdf\xdd\xb4\xab\x16\xf1\x0eJ\xab0\xd8\xc5{\x13\x82v\x9cz\x13\x8fZ\xbfo\xf0\xfe\xee\xe2R.$\xc88\xe1A# \xe4\x1c\x1e3\x9e\xff\x00OJ\xf6\xe8\xac\xec\xee\xf4\xe2\xf6\x13"Es\x16\xe8nm\xf6?\x0c>WBAS\xd4\x11\x90A\xf7\x15p[\xc6\xa7!\x14\x1fP(\x03\xc2\xe6\xf0\x14\x96\x16\xd1\xcf{q+\xae\xe5F\xf2\xe3<\xb1 gh\xc9\x03\'\xa9\xe0\x0c\xe4\xf0Mr\xb70\xac\x17\x0c\x9b\x1e<\x00v\xb9\x05\x97\xd8\xe0\x91\x91\xecM}\x0f\xe2X\x16}\x1ex^\x14\x969\x14\xa3\xa3\xa8ee \x82\x08=x\xaf\x07\xd6\xcb\x9df\xe5\xa6,]\xd8\x92X\xe4\x93\xeah\x02\x8e\xd8\xb6\x02\x16O4\x92\x19\x8b\xe5J\xf6\xc2\xe3\x83\x9d\xd99\xe7#\xd3\x95\xb7\x86Y\xee"\x8a$i$w\n\xaa\xa3%\x89<\x00;\x9a\xb1\xf6G[\x91\x0c[.\xda`c\x87\xca%\xb7\x1d\xe0\x06\n0\xc1\x8e\xd2\x00a\xd1\xb3\x8eT\x8a\xac$\x0c\xae\xa4\x05^\xab\x8eO#\xbf\xe7@\x1e\xe9\xe1\x94\x98\xcdm!\x91Lb\xdbk+),O\xcb\xb7\r\x9e\x00\x00\x8c`\xe7#\x91\x8ez\xde\x7f\xbb\\\x9f\x84\xae|\xfb\x1bI\x82`<`c\xd2\xba\xcd\xd4\x01\xe4\x1f\x02\xce|\x12\x9c\x8e%n\xfe\xf5\xeb\xcb\xf7Ex7\xc0\xfdY\x93\xc3\x97\x0b4\xd0Aki)\x92Y%\xe0\x08\xf6\xb6y\xc8\x0b\x83\xb4\xee9\x18R1\xceG\xbc\xc4~A@\x1e]\xf1\x02\x12\xf7@4(\xf1\x96\xdcw\xf6\x03$0\x189!\x80\xf4\xf5\xcf\x18\xaf0t\x8dau>\\\xa6h\xc88.\xad\t\xdd\xea1\xf3`{\x8c7\xafOb\xf1\xd8\xb5\x8e\xea7e\x80O*\x98\x91\x9dWq\xcf\xccUI\xe7\x90\x80\x9cv_A^=>\x16\xeaU\xed\xb8\xd0\x02[\x0b\xaf\xb7@l\xcc\x9e~\xf1\xe5\x88\x81\xf37\xe4m\xdaG9\xcf\xa79\xc5GO\x8d\x15\xd8\x86\x91c\x1bI\xdc\xd9\xc1 \x12\x07\x00\xf2z\x0e\xd9<\xe0sRFm\x89Eo5\x99\x94\x028P\x8d\xbf\xf1\xdc6\xff\x00\xbb\xc9\xf4\x1c\x80$s\x13\xa85\xcc\xac\x8b\xc9\x91U-\xd4\xae\xfeH\x1b8]\xa4\xe0c\x18\x03<\x1e\x95\\\xb2\x8e\xf5\xdf\xd9\xf8.\x1b\xf8\x83Z\xc2\x81\x81\\\x19\t;\x86FGQ\x83\x8c\xf2N\x07^z\x1d\xe3\xe0\x8b\x87\x8e\x1f&8\xa2U9a\xb0|\xc3\x04`\xe4{\xe7\x8c\x1c\x81\xdb \x80y*\xf9\x8d\x19D\xb5\xde\xacA\xf3F\xed\xcb\x8f\xe1\x03\xa6\x0ers\xcf\xca1\xd4\xd4\x91Y\xdd\xce\xc5b\xb5\x95\xb1\xd7\xe5\xc5{ \xf0\x9aYY\xc9=\xd5\xc40C\x12\x97\x91\xd8\x84DP2X\x93\xc0\x00s\x9a\xb6\xbe\n\x8at\x92\x1b\x96Y"u*\xc8FC\x03\xd4\x11\xdcP\x07\x8c\xda\xe8Z\xa4\xd3,7\x16\xe6\xd8\xb3}\xf9\x81P\xaaO\xca\xc7\x198\xc1\x07\x8e\xbe\x9d\xaa\xcaxWR|\x90\xa9\xb5FX\x86\'h\xce2\x7f\x12?:\xf6K/\x0bZ\x17\x13\xaf\x98B\xc8\xcb\x89b+\x92\xacA\xe1\x87##\x83\xd0\x8c\x11\x90A\xad(\xbc9c\x13\x02#\xcf\xd6\x80<8xR\xe4JCN\x81\x00\x18!\t$\xf3\x9e1\xf4\xe7=\xcfLs}| \xc5\x03\x03t\xc0\xf7\x0b\xc5{\x05\xd4P6\x9f\x05\xf6\x9a\xb6\xd7\x11\xc9\xb5\xa3\xc2\xb4\x898q\x84\xc3\xa0;T\xb3!/\xb5\x80P\xc7\x18\xe4i[[\x91\x11\x12\xdb\xc5\x1b\x07`\x026\xe1\xb7q\xdar@\xe4\xae\t\x18\xe0\x922q\x92\x01\xe3\x16\xde\x05\x96h\xf7-\x9c\xd2\x7f\xb4\xc7\x06\xafi\xdf\x0f/$\x03\xcd\xb1H\x9c\xa8,\x19\xcb(=\xc0$\x0c\xfeB\xbdw\xca\x90\\!W\x8c@\x11\x83\xa6\xc3\xb8\xb6F\xd2\x1b8\x00\r\xd9\x189\xc8\xe4c\x99\xb6\x81\xda\x80<\xc2\xdf\xe1\xcc\x8b(,\xb1 \xee@\xa9\x1b\xe1\xac\xe7\xc8\x1f\xda\x1f+&\xcb\xb4H\xf6\xa4\xc3\x83\xc0\xc9#\xe6U#$\x8e\xa3\x04\xe1\x87\xa1Z\t\x1a#<\xbez4\xdbd\xf2&\xd9\x98>P\ne88 \x93\xcbrN\x0e0\x02\xed\xb6\xb4\x9f)\x06\xc9.\xe5\xf9\xda(I\xde\xe1>\xf3\x908\xf9P\r\xcd\xe8\xab\x9c\xe0P\x07\x16\xbe\x00\xd3,\x04Q\xc3\xf6\x85y\x99\xd02olnV,\x18\x8c\xedR7\x0ep9\x0b\xdc\n\xb8|\x1bgk\xa7\xb6\xdbS;F\xac\xeb\x1cj\xbb\xe48\xfb\xa0\xb1\x03\'\x18\x19 s\xdb\x9a\xec*8fY\xe3.\x81\xc0\x0e\xc9\xf3\xc6\xc8r\xacT\xf0\xc0\x1cdpz\x11\x822\x084\x01\x83\x0f\x84l\x13fD\x8c\x100T\xdeB\r\xdbrv\xe7\x04\xfc\xa3\x9cd~&\xaeC\xe1\xdb\x08\xba[\xaezs\xde\xb5\xaa\x84\x1a\x9d\xbd\xc6\xfb\x9b{\xd8.\xec\xcf\x93\x1a\x8bT2\xb2\xbb\xe0\x82Y\t\xf9J\xc9\x13t\x1bW,IS\xc0\x05c\xa2Y\xda\xc0g6f\xf6{v\x92hAH\xfc\xcd\xc7v\x15\t\xda\xa0\xedb\x80\x928\xeay$\xe8Gd\x88\xf2\xb31\x90;\xeeUe\\F6\x81\xb5p\x01\xc6A<\xe4\xe5\x8f8\xc0\x12\xc7\x040\xbc\xcf\x14Q\xa3\xcc\xfb\xe5eP\x0b\xb6\xd0\xb9oS\xb5Td\xf6\x00v\xaa\xf3\xcdr5K;xDb\x06I$\x99\xda7bB\xed\x01T\x81\xb5I.\x0eX\xe7\n@S\x92\xc8\x01h"\x8c`t\xa8\xe3IC\xcd\xe6<l\x85\xf3\x10T*Uv\x8e\x18\xe4\xee;\xb7\x1c\x808 c\x8c\x98o \xd3E\xc5\xad\xfd\xecV\x9e}\xbb\xec\xb7\xb8\x99WtM!\t\x84c\xc8,H\\\x0e\xb9\x02\xad\xe2\x80)\x18ZR\x96\xd7\xb6\xe9r\x85\x9aa(\x8dDhR@\xd1\x02\xac\xc4\xef\x1c\x10\xc0c1\x93\xf2\xfc\xa2\xac\xc0fkx\x9a\xe28\xe3\x9c\xa02$n]U\xb1\xc8\x0c@$g\xbe\x06}\x05IQ\xc12\xdc\xdb\xc5:\x07\t"\x07Q$l\x8c\x01\x19\xe5X\x02\xa7\xd8\x80Gz\x00\x92\xab\xd9\xf9RDn\xa2\xf3\xc2\xddm\x9b\x13o\x04e@\x03c\xf3\x1f\x00ep9\xc9#$\xd5\x8ad\x86`\xf1\x08\xa3\x8d\x90\xbe%,\xe5J\xae\xd3\xca\x8c\x1d\xc7v\xd1\x83\x8e\t9\xe3\x04\x00\x84\xcc\xd1\x93<q\xa3\xef`\x029a\xb7q\xdar@\xe4\xae\t\x18\xe0\x922q\x93^\xc9\x15n\xb5\x12\xa9\x1a\x96\xb8\x05\x8a\xdb4E\x8f\x95\x18\xcb1\xe2S\x80\x06\xf1\xc0\x00/T5r\xab\xde%\xc3\xc0\xa2\xd5\xf6I\xe6\xc6I\xdc\x17\xe4\x0e\xa5\xc6J\xb7U\xdc1\x8ez\x02\xa7\xe6\x00\x16*;\x832\xdb\xca\xd6\xe9\x1c\x93\x84&4\x91\xca+68\x05\x80$\x0c\xf7\xc1\xc7\xa1\xa8\xcd\xc4\xa2\xcei\xc5\x94\xe6H\xf7\xed\x80\x14\xdf&\xd2@\xdawm\xf9\xb0\x08\xc9\x1fxgi\xc8\x05\xc8\xb7\x83u\xfc\xb0o\x92\xde\'\x01\xd2\x13$\x81\x0e\x0b*\x85\x05\x8ev/\xca:\x95\x1c\x13\x8a\x00\xa5\xaf!}&\xe1VW\x89\x99\x08\x12 \x1b\x90\xe3\xa8\xc8##\xdc\x11\xed^\x0f\xe2\t\x9e-zibvVe)\x958;X\x10G\xd0\x82A\xf65\xed\x9a\xed\xcc\x96\xb0j\x0fu3C\x07\x90Z>s\x12\xc6\x80ow}\x80#\x12\xe4m,AT\x04c\xe7\xc7\x87\xeb\xb3J\xda\x94\xb01\x93\xcaO\x9d\x01?.\xe3\xc1\xc0\xcf_\x95s\xc7\xa7\xa7\x00\x18\xe0)i"+(\xdf\x9f\x9bq\xc1\x18\x03 \x83\xf2\xf5\xf69\x04\xfb\xd5\x94a\x05\xb3D\x8a\x8a\x8c\x9b\x02\xec\x07h\x04\x11\xb7\x8f\x97\xa0\xe9\x8e2:\x13K\x11TH\xa6\xc43G4e\x93\x12n\x189\x00\xfc\xa7 \x83\xc8\x07\xd8\xe0\x83\xcb\x12 \xf2\x96\x9avT\n\xdbBF\x0f8\xe39>\xa3\xa8\xc6\x01\xe8H\xa0\x0fv\xf0|-\x16\x89\xa7\x87]\xb8\x88\x11\x95\xc7\x07\x91\xfc\xeb\xa7\xaeW\xc0\x97-w\xe1\xf8\x18\xc6\xaa\x89\x85B\x1b%\x86\x07$``\xe7#\x1c\xf0\x01\xcf8\x17\x7f\xe11\xd2\xff\x00\xe7\xd7\\\xff\x00\xc1\x15\xef\xff\x00\x19\xa0\x0f"\xf8\x15q\x1c\xd6\x9a\x8bGk\r\xb8k\x82\xe1c.B\x83\x8c \xdc\xc4\xe1{g\'\x9eI\xafv\xb5\x85#\xf3]L\x84\xca\xc1\xdbt\x8c\xc0\x1d\xa1~PN\x14aG\x03\x039=I\'\xe7?\x80\xda\x86\xdb\x9b\xdb-\x9e\x8f\xbb5\xf4\x14\xf0i\xa6\x01{\xa8\xc5k\xb2\xd5\x0b\xf9\xf7\n\xb8\x85C,\x84\xeeo\xba\x03F\x8d\x9fTS\xd8P\x07+\xe3\x8f\xf5\xaa=\xab\xc6\xa7\'\xed\x13o@\x1b\xcc8\xc1\xcf\x19\xe3\xf4\xafm\xf1\xc2\xc6\xcd\x12l\x7f5\x91\x98\x10\x84\xae\x063\x96\xc6\x01\xe4`\x13\x93\xce:\x1cx\xe6\xae\x86\rNE\xc2\xfc\xc1[\xe5 \x8eF{w\xe7\xa7n\x94\x01I\xa7k\xa77\x0f1\x99\xa5;\xcc\xa5\xb7\x17\'\x9c\xe7\xbez\xe6\x8d\xa0\x94a\xc8\x0c2i(f\xc2\x91\xd8\x8a\x00\xf6\x7f\x06\x0c\xc8\xb3\xb4\xd9A\x16\xc4E,9\',O\xcd\xb5\xba.8\xc8\xc3s\xf3\x11]\x9d\xd1\x9f\xc9\tj\x9f\xbd\x97r,\xac\x81\x92\x13\xb4\x90\xee\xbb\x94\xb2\xe4\x01\x859\xe4t\x19#\xcf|\x15\x10\x91l\x9d\xd2\x17X\x97z\x97\x8f.\xaf\x8d\xa1\x95\xb3\x85\xf9Y\xc1\xe3\'wP2\x0f\xa4[\x89\x96"\'tw\xdc\xc4\x14B\xa3n\xe3\xb4`\x93\xc8\\\x02s\xc9\x04\xe0g\x00\x006\xe8n\x12r_z# \x02F\x0b\x82A9\\\xe0\x9f\x94`\x91\x91\xce1\x93\x95x\x12G\x89\xd8\xb81>\xf5\xdb#(\'i_\x98\x03\x86\x18c\xc1\xc8\xce\x0fP\x08\x92\xaa&\x9e"\xb7\xbbH\xa7\x929\xee\x9d\xdd\xeeR8\xc4\x81\x88\xc2\x9f\xbb\xb4\x95P\xaa\x0b\x03\xc2.ww\x00\xb9U\xe1o2\xf2\xe5\x84\xb3\x95M\xb1\x18\x9e=\xa8\xac\x06\xe2\xcaJ\x82\xd9\x0e\xa0\x90J\xfc\xb8\x18!\xa8\xfbV\xd9\xfc\xa9-\xe7M\xd2\xf9Q\xbe\xcd\xca\xff\x00&\xfd\xd9\\\xed^\n\xe5\xf6\xfc\xc3\x1d\xd72Bfd&x\xe3G\xde\xc0\x04r\xc3n\xe3\xb4\xe4\x81\xc9\\\x12;\x12FN2@\t\x04\xc5\xe21I\x1a\xa0|\xca\x19\x0b\x16]\xa7\x859\x1bN\xed\xa7\'<\x021\xceD\x95\x19\x82\x16\xb8K\x86\x8a3:#"HTnUb\x0b\x00z\x80J\xa9#\xbe\xd1\xe9RP\x01E\x14P\x05g\x9b\xed\x11]Ees\x00\xba\x8b1\x92\xc3\xcc\x11HT2\xefPA\xe8\xca\xdbr\t\x04r3\x9a\x92h!\xb9@\x93\xc5\x1c\xa8\x1d\\+\xa8`\x19X2\x9e{\x86\x00\x83\xd8\x80i\x96\xf7\xb6\xf7s\xdd\xc3\x04\x9b\xa4\xb4\x94C8\xdaF\xc7(\xb2\x01\xcf_\x95\xd4\xf1\xeb\xeb\x9aI~\xc7u+Y\xcb\xe4M$^\\\xed\x0b\xe1\x8a|\xc4\xc6\xe5{|\xc8H>\xa9\xc7"\x80\x0b\x19^[8\xcc\xad\xbee\xccr8\x81\xa1\x0c\xeaJ\xb1TbHRA#\x93\x91\x82\t\x04\x13!\x85Z\xe1\'%\xf7\xa22\x00$`\xb8b\t\xca\xe7\x04\xfc\xa3\x04\x8c\x8eq\x8c\x9c\xc9K@\x19\xf6Vw\x0bt\xd77\xa6\t$_1 m\x81\xa4\x8d\x1aVb7\x80\xbf)Q\x0f\xcb\xb7\x82\x9c\xb3\x9c5hU{\xeb\x1b}J\xceKK\xb8\xfc\xdby0$\x8c\xb1\x01\xc0 \xedluS\x8c\x15<0$\x10A"\xa41\xb1\xb8ID\xd2\x04Te1\x00\xbbX\x920\xc7\x8c\xe4`\x81\x82\x07\xccr\x0f\x18\x00\x92\xab\xa1\xb7\x82\xf1\xa2\xf3\xff\x00\xd2.s0\x89\xe6,HP\x8a\xc5T\x9e\x14e3\xb4\x01\x96\xc9\xe5\xb2lTbxZ\xe1\xed\xd6X\xcc\xe8\x8a\xef\x18a\xb9U\x89\nH\xea\x01*\xc0\x1e\xfbO\xa5\x00g\xc3n\xd1\xea\x86+\xc6\x92\xf0ok\xbbI\xa6\x8dI\x81\xb9FE+\x1a\xaa\x00\xae\x02\x92\xc5\xd8<\x83\xa2\x9a\xb947\x12^[I\x1d\xd7\x95o\x1e\xe3,B0L\xa4\x8c(,z(\xc9$\x01\x92B\xf2\x00!\xa4\x80\xcc\xd6\xf15\xc4q\xc79@dH\xdc\xba\xabc\x90\x18\x80H\xcf|\x0c\xfa\n\x92\x80+\xc5}g<\xab\x14WpI#y\x9bQ$\x04\x9f-\x82I\x81\xfe\xcb\x10\xa7\xd0\x9c\x1ejBf\xfbB*\xa2\x18\n1w.C\x06\xc8\xda\x02\xe3\x04\x11\xbb\'#\x18\x1c\x1c\xf0N&kyV\xdeH\xe3\x9c\xa1\x11\xbc\x88]U\xb1\xc1*\x08$g\xb6F}E\x13\t\x99\x00\x82H\xd1\xf7\xa9%\xd0\xb0\xdb\xb8n\x18\x04rW \x1c\xf0H88\xc1\x00\xafy,?h\xb5\xb4\x9fz\t\xdf1\xc8\xb3\x08\xf3"\x10\xe18`\xccHVl\x00ATp\xdcplO\n\xdc\xdb\xcb\x03\x99\x02H\x85\x18\xc7##\x00F8e \xa9\xf7\x04\x11\xda\xa4\xa8\xcc\xf0\xad\xc2[\xb4\xb1\x89\xdd\x19\xd22\xc3s*\x90\x18\x81\xd4\x80YA=\xb7\x0fZ\x00\xaf$kh\x90\xb2\xcdv\xa8.72\xa8i\x8c\x86F#kd3\x04\x0c\xe0\xf1\x80\xa1G!\x01\x15$\xcd\xe5\xde[1\x96p\xaf\xba!\x12G\xb9\x19\x88\xdc\x19\x88RW\x01\x18\x02H_\x9b\x07$\xad\x1b\x13O\xd3\xbc\xbb;<\xc7o\x16!\xb5\xb7\n\x99\n>T@HQ\xd0\x01\x92\x00\xf5\x02\xa4\x9exmm\xe5\xb8\xb8\x968`\x89\x0b\xc9$\x8c\x15Q@\xc9$\x9e\x00\x03\x9c\xd0\x014\xcb\x02\x07q!\x05\xd5>H\xd9\xceY\x82\x8e\x14\x13\x8c\x9eO@2N\x00&\xa4\xa8\xe4\x85ex]\x8c\x80\xc4\xfb\xd7l\x8c\xa0\x9d\xa5~`\x0e\x18a\x8f\x07#8=@ \xb8\x91\xa1\xb7\x96T\x86I\xdd\x10\xb2\xc5\x19P\xce@\xfb\xa3q\x03\'\xa7$\x0fR(\x02\x1b\xec\x1b9x\x07\xe5=k\xc0|Kn&\xd7a\x8b\x11\xef\'\x08\\\x85\x00\x9e3\x93\xc0\xeb\xd4\xf6\xaf|\xbfo\xf49\x7f\xdd5\xf3\xef\x89\xfc\xbf\xed1q\x02+\x0b\x84Yw\xc7\x8f\xde\x06\x03\x0f\x9e\xe3n\xd3\x9e\xe0\x0cg\x8a\x00\xa1p\x90f9m\xa5;\x18\xb8h$\\\xbc|\xfc\xbf8\xc0a\x8e\xf8\x079\xe0\x00\x0b4\x92\x10\xfd)\xb6^Tq\xbcs\'\x1b\n\xa3d\xbb)\x1c\xa9\xe4\x8eN\x009\xce\x03\x1e\xa4\nV\x05\xa3#\xbfj\x00\xf6\xcf\x87\xac\xd1xf\xcd\x04\x12>\xf7*\xcc\xa5q\x18\xdaN\xe3\x92\x0e2\x00\xe3\',8\xc6H\xed1\\\x8f\xc3\xc9\x03\xf8^\xdf\xf7\x88\xec\t\x1f*\xed\xe3<\x0cd\xf4\x18\x19\xeerx\xe8:\xea\x00\xf9_\xe0\x841\\j\xfa\x84\x13\xc6\x92\xc3,;$\x8eE\x0c\xae\xa7\x82\x08<\x10Gj\xfab\xda\xe4\xcf\xa5\x8b\x9d<\xc1rd\x8b\xcc\xb7>n#\x97#+\xf3\x80\xd8S\xc7 \x1e\x0epk\xe6\x0f\x82\xfa\xa3A\xe2O\xb0\x18\xd0\xa3\x86`\xc1\x140\'\x19\xcbc$|\xa3\x00\x9c\x0eq\x8c\x9c\xfd;\xa7\xc8\x97^]\xc4\x17{\xe2@\xf14q\x95(\xce\x18\x03\x93\x8c\xeeR\xac\xb8\x04ul\x82@\xc0\x06G\x8b\xd9\xa3\x81\x19\x18\xa9\xc3\x0c\x83\x8e\x08\xc1\xaf\x10\xd5\xf7\x7fhI\x8e\x01\xc5{\xaf\x8c\x121\xa6\t_\x7f\x0c\xa86!c\x96 \x0e\x00\xe9\xcf\'\xa0\x1c\x9e\x05x\xde\xb5(\xb4\x99b\xf2\x96X\xdeT\x91\xd5\xd9\xc0m\xbb\x86\x08V\x1c\x1d\xc7\x9e\xa3\xb1\x199\x00\xc4\x8a\xd9\xe7\xdc\xd1\xb9g@\x7fr\xaaIe\xc1f~\x07E\ns\xc8\xeb\xd0\xe0\x90\xc9a\x928\xd5\x9d\x19VE\xde\x84\x8c\x06\x19##\xd4d\x11\xf5\x06\x9a\xa8\xb1\xa0DP\xaa\xa3\n\xa0`\x01SD%0\xdc\x18\xa3\xde\xa1\x07\x98\xdeXm\x8b\xb9y\xce>^v\x8c\x8cu\xc7|\x10\x0fY\xf03\x83en\xbf.\x00\x07\xa0\xcf\xe7\xf8W\xa1\xcb#E\x18d\x85\xe6%\xd5v\xa1P@,\x01o\x98\x81\x80\x0eO|\x03\x80N\x01\xf2_\x04Om\x19\x89,\xa1\x92\x1bn\x168\xe4\x909Q\xe9\x90\xaa?A^\xb9\x17(3\xe9@\x0f\xa8\xe4\x91\x91\xe1U\x86I\x03\xbe\xd6e+\x88\xc6\xd2w6H8\xc8\x03\x8c\x9c\xb0\xe3\x19"J\x8e\x19\x96t.\x82@\x03\xb2|\xf1\xb2\x1c\xab\x15<0\x07\x19\x1c\x1e\x84`\x8c\x82\r\x00\x04\xcd\xf6\x84U\x8e3\x01F.\xe5\xc8`\xd9\x1b@\\`\x827d\xe4c\x03\x83\x9e)\xdf\\\xdcZ\xdeA1\x9a\x084\xc8\xa2\x96[\xd9fA\x85\n\x06\xdf\x9c\xc8\xbb:\xb3\x12U\x86\x14\xe4\xaf\x19\xb8`\x85\xae\x12\xe1\xa2\x8c\xce\x88\xc8\x92\x15\x1b\x95X\x82\xc0\x1e\xa0\x12\xaaH\xef\xb4zU9!\xb3\x9f\xec/\xaa\xdbX\x8b\xe6\xc2F\xacD\x98\x93\x89J\xc6\xcc\x01l\x18\x83\xf4\x07\xf7a\xb06\xf0\x01b\xcf\xca\x8e#k\x17\x9eV\xd7l9\x9by-\x85\x04\x1d\xef\xcc\x9c\x11\x96\xc9\xe7 \x9c\x83D\xd7^_\xda\x16;y\xe7\x9a\x18\x84\xa24Ly\x99\xdd\x85Vl!bT\xf1\xbb\x8c\x82p\x084Y\xf9\xaf\x11\x9e_=\x1am\xb2y\x13l\xcd\xbf\xca\x01L\xa7\x07\x04\x12ynI\xc1\xc6\x00\x92\x18\xda$*\xf3I1.\xcd\xb9\xc2\x82\x01bB\xfc\xa0\x0c\x00p;\xe0\x0c\x92rH\x00$cp\xf1\x18d\x08\xa8\xac%%v\xb1$\xe5G9\xc8\xc0\' \x0f\x98`\x9epM#D\x81\x92\x19&%\xd5v\xa1P@,\x01o\x98\x81\x80\x0eO|\x03\x80N\x01 \x82\x1b[x\xad\xed\xe2\x8e\x18"@\x91\xc7\x1a\x85TP0\x00\x03\x80\x00\xe3\x15%\x00W\xb7{\x86\x9e\xecN\x9bcY@\x80\xed\x03rlRNC\x1c\xfc\xc5\x86H^\x98\xc6\x00f\xb1E\x14\x00TpL\xb76\xf1N\x82@\x92 u\x12F\xc8\xc0\x11\x9eU\x80*}\x88\x04w\xa9(\xa0\x08\xe6\x132\x01\x04\x91\xa3\xefRK\xa1a\xb7p\xdc0\x08\xe4\xae@=\x89\x07\x07\x181\xc3\xe6\xb5\xe5\xcb\xbf\x9e\xb1\xae\xd8\xd1\x1ff\xc6\xc0\xdc]q\xf3s\xbbi\xdc\x7f\xe5\x9f\x00\x0eZA<-p\xf6\xeb,ftEw\x8c0\xdc\xaa\xc4\x85$u\x00\x95`\x0f}\xa7\xd2\xab\xe9\xba\x9d\xb6\xadn\xd7VRG=\xa6\xf2\xb1\\E*I\x1c\xc0\x01\x96B\xacx\r\x959\xc1\xca\x9e1\x82@.Us\xf6=N\xceh\x8f\x91wk&\xf8%C\x89\x11\xb0J:0\xe8pC)\x07\xd0\x83E\xd2\\I\xe4%\xbb\xf9jeS+\x86\x01\x82\x0ep\x01V\r\xb8\x80\xa4q\xf2\xb3\x10A\x02\xa4\x866\x89\n\xbc\xd2LK\xb3np\xa0\x80X\x90\xbf(\x03\x00\x1c\x0e\xf8\x03$\x9c\x92\x00@fkx\x9a\xe28\xe3\x9c\xa02$n]U\xb1\xc8\x0c@$g\xbe\x06}\x05G}\x7fg\xa6Y\xc9y\x7fw\x05\xa5\xacx\xdf4\xf2\x08\xd1r@\x19c\xc0\xc9 ~5b\xab\xc4\xdeD\xabm$\xb3\xcd$\x9ed\xaa\xef\x1f\x01w\x0f\x94\xb2\xa8Q\x8d\xe0\x00~b\x17?1\x0ch\x02?\xb1\xad\x9a\\\xcb\xa7\xdb\xc6\'tb\xb0\x99Z8\x99\xf7;\xe4\x80\x08R\xcc\xecY\xc2\x92s\x93\xbb\x02\xa4{\xfb8\xef\x16\xcd\xee\xe0[\xa7\xc6\xd8L\x80;d9\x18^\xbc\x88\xe4#\xfd\xc6\xf44M\x13\x9b\xcbi\xa3]\xdbwG&\xe9\xd9B\xa1\x19\xc8@\n\xbbnT\x038\xc0-\x83\xd41r\xf7\x07t6\xc9\xb6G\x89\xca\\:\x87\x8e7\x18\n\x19w+6s\x9c\x0e\xcarT\xe3 \x16*8diP\xb3\xc3$$;.\xd7*I\x01\x88\r\xf2\x920@\xc8\xef\x822\x01\xc8\x12Ts\x19\x960`\x8e7}\xea\x08w*6\xee\x1b\x8e@<\x85\xc9\x03\xb9\x00dg \x02J\xaf\n$W\x97";?+\xcc\xdb,\x93\x80\xa0L\xe4m\xe7\x07q`\xa8\x80\x92:\x15\x00\x9c\x10,Tfe[\x84\x80\x87\xde\xe8\xce\x08\x8d\x8a\xe1H\x07-\x8c\x03\xf3\x0c\x02ry\xc6pp\x00\x017\xda\x1d\x9aH\xcc\x05\x14"\x04!\x83d\xee%\xb3\x82\x08\xdb\x81\x81\x8c\x1eNx\x92\x8a\x8eC0x\x84Q\xc6\xc8_\x12\x96r\xa5Wi\xe5F\x0e\xe3\xbbh\xc1\xc7\x04\x9c\xf1\x82\x01\x1d\xd2\xa9\xb7p@#\x07\x8a\xf0\x9f\x19\x0bq:\x15\x89\xc4\xdb\x88\xdf\xbcc\x199\xe3\x1f\xee\xf7\xecz\xe7\x8fx\x9cf\x17\xfaW\x84\xf8\xc9#\xfe\xd2E\x95\xd9\x14o9U\xdcs\x83\x81\xd4pN\x01=\x878=(\x03\x97\x01\xd6B\x8e\x9b~P\xc0\x87V\x04\x1eG\xdd\'\x9c`\xe0\xf23\x83\x82\x08\x16\x90B\x9b\x0c\x8f\xe6\x07\x8d\x8e\xd4%J7 d\x91\x8e\xa0\x1e3\x90q\x90zA\x19\x8c#\xeeV$\xaf\xc8C`\x03\x91\xc9\xe3\x91\x8c\x8cq\xc9\x07<`\xca\xeb$\x96H\x8f!0\xf9\x8cQ\x04\xbc\x86\xf9I;A\xc8\xce\x14g\xa1\xc69\xc1\xa0\x0ff\xf8h\xb8\xf0\xdc\x7fS]\xb5q\x7f\r\xda3\xe1x\x15\x18\x16\x04\x86\x03\xb1\xae\xd2\x80>G\xf8D\xefw\xf1\x05\xaeZHb\x94\xab\xcd\x85Q\x12\x96\xceH@\xa0*\xf5$\x0e\x06\x06\x078\x07\xea\xbb\x1f\xf5+_ \xfc.\xbdK?\x1bZ\x17\x04\xf9\x9f \xc7\xad}y\xa7\x1c\xc2\xbfJ\x00\xa9\xe2\x7f\xf9\x04\xb7\xd6\xbc\'\xc4s\'\xda\xb76HL\x92\x00$\xfeC\xad{\xce\xbe!\x9e\xc9\xadL\xf1\xac\xef\x1bH\x91\x96\x1b\x99T\x80\xc4\x0e\xa4\x02\xca\t\xed\xb8z\xd7\x84k\xbaq\xfb[y%L\x8f&\xd5\xf3d\x08\xb9=2\xc7\x85\x1e\xe7\xa5\x00d\xcb\x1bE+\xc4\xc5K#\x15%\x182\xe4z\x11\x90G\xb8\xe2\x99\x9cq\xda\xa4\xb5kWb&g1\x93\xb7\xcc\x88\x82P\x86\x01\x8e\x0f\xde\xc0\x0c1\x95\xe7\x1c\xf1\x8a\x8eHe\xb4\x95\xa1x\x9d\x1d\x1d\x84\xab!!\x90\xf3\x91\x83\xdf<c\x8c~\x18\xa0\x0e\xef\xc1\xb7,\xbb^W\x92Y\xa7\xb8%\xa5\x95\xf7\x93\x96,\xc4\x96\xc9\xc98\xe7>\xa3\xbdz\xec\xb0J\xf7z}\xcd\xbcV\x84\xa32O,\xaa|\xc5\x85\x90\x92##\xa12,$\x83\xc1\n{\x81^\x13\xe1\xab\x80D\x91\x87\x1b\x91\xd1\xb6\xe7\x95\x04\xf5\xc7\xbe\x0f\xe5^\xf3\x0c)s\xa5\x88]\xa4\t,[\x18\xc7##\x00F\x0e\x19H*}\xc1\x04v\xa0\x0b\x10\xc6\xd1!W\x9aI\x89vm\xce\x14\x10\x0b\x12\x17\xe5\x00`\x03\x81\xdf\x00d\x93\x92H\xe4gy\x95\xa1\x920\x8f\xb5Y\x8a\xe2A\xb4\x1d\xcb\x82N2H\xe7\x07*x\xc6\t\x92\xab\xde\x0bs\x02\x8b\xa8<\xe8\xfc\xd8\xc8_$\xc9\x87\xde\xbb\x1b\x00\x1ca\xb6\xb6\xef\xe1\xc6\xe2@\x19\x00\x16*\x99\x86\x0b\xcb\xf4\x9c\x9b\xb5\x96\xc9\xd9\x00\x12K\x14lY\x019\\\x84\x94`\x8c\x12\x18\x03\x9c`\x83R8\xb7\xb1\x8a\xea\xe8A\xb7vf\x9c\xc3\tg\x90\x85\x03;T\x16v\xda\xaa\x06\x01\'\x00\x0e\xd5$\x16\xf0\xda\xdb\xc5oo\x14p\xc1\x12\x04\x8e8\xd4*\xa2\x81\x80\x00\x1c\x00\x07\x18\xa0\x08\xd9\xbe\xd5\xe6E\x14\xb3\xc0\xd0\xca\x81\x9cG\x8d\xd8\xda\xe4\x02\xeaC)\x07i+\xea\xc0\x10\xc3\x82\x14\x8e\xd3\xec\xf6V\xd6~]\xaaDB\x18\x82\xacq\x05\xda\x156\xe7# \xf1\x81\x80\x14\xe4\x8e3\x1c\x9b<\xb8\xa7\xbe\xb5\x8c\xc9\x15\xc7\xee|\xa4i\x8a\x16c\x1a\xb8\xf9r\xa4\xab\xfc\xc7\x18P\xcd\x92T\x167(\x02\xbd\xf4\xd7\x10Y\xc9-\xad\xaf\xda\xa6L\x11\x08\x90!q\x91\xb8)<n\xc6p\t\x00\x9c\x02T\x1c\x8b\x15\x1ck0\x92S,\x91\xb2\x17\xccAP\xa9U\xda8c\x93\xb8\xee\xdcr1\xc1\x03\x1cd\xac\xd3$\x08\x1d\xc4\x84\x17T\xf9#g9f\n8PN2y=\x00\xc98\x00\x9a\x00}G\x1cl\x8f+4\xcf w\xdc\xaa\xc1q\x18\xda\x06\xd5\xc0\x07\x19\x04\xf3\x93\x96<\xe3\x002\xe6\xe7\xc8\x9e\xd2=\xd0\x0f>S\x1e%\x97c\x1f\x91\x9b\xe4\x18;\xdb\xe5\xfb\xbc|\xbb\x8e~\\\x19-\xe6K\x8bx\xa7@\xe1$@\xea$\x8d\x91\x80#<\xab\x00T\xfb\x10\x08\xef@\x0f\xa8\xe4\x8d\xdeH\x99f\x921\x1b\xeeeP\xb8\x90m#kd\x13\x8c\x90x\xc1\xca\x8eq\x90c\xb7K\x88\xe5\x91%\x7f6\x1f\xbe\x92\xbb\r\xe4\xb31(T(\x01Tl\x00\xe4\x93\xdf\x91\x96\xb3@\x11)\x9b\xed\x0e\xad\x1cb\x00\x8aQ\xc3\x92\xc5\xb2w\x02\xb8\xc0\x00m\xc1\xc9\xceO\x03\x1c\x90B\xb6\xd6\xf1@\x85\xcaF\x81\x14\xc9#;\x10\x069f$\xb1\xf7$\x93\xde\x94\xb4\xdfhEX\xe30\x14b\xee\\\x86\r\x91\xb4\x05\xc6\x08#vNF089\xe0\x84L\xa8D\xf2F\xef\xbd\x88(\x85F\xdd\xc7h\xc1\'\x90\xb8\x04\xf7 \x9c\x0c\xe0\x00$&fBg\x8e4}\xec\x00G,6\xee;NH\x1c\x95\xc1#\xb1$d\xe3$Y\x1c\xdc<F\x19\x02*+\tI]\xacI9Q\xcer0\t\xc8\x03\xe6\x18\'\x9cA\xfd\xa2\xac\x96\xcd\r\xb5\xdc\xa6\xe1\x16E_!\x90\xaa\x16@K\xef\xda\x10\x80\xe0\x95l1\n\xd8RA\x14\xff\x00\xb7[\x9d?\xed\xf1I\xe7\xda\x98\xbc\xe5\x92\xddL\xbebc \xa0L\x97\xc8\xe9\xb79\xed\x9a\x00\xb1Py\xb7\x07Q\xf2E\xae-D[\xda\xe1\xa4\x1c\xb98\x08\xab\xc98\x00\x92N1\x95\xc6\xec\xb6\xdb\x15\x12\xc6\xe2\xe1\xe53HQ\x91TDB\xedR\t\xcb\x0e3\x93\x90\x0eI\x1f(\xc0\x1c\xe4\x00\x92xa\x92$\x96X\xd1\xe6}\x91+0\x05\xdbil/\xa9\xda\xacp;\x02{R\xc7\nD\xf3:\x99\t\x95\xf7\xb6\xe9\x19\x80;B\xfc\xa0\x9c(\xc2\x8e\x06\x06rz\x92I\x04o\x12\x15y\x9eb]\x9bs\x85\x04\x02\xc4\x85\xf9@\x18\x00\xe0w\xc0\x19$\xe4\x92e\x99\x90\x08$\x8d\x1fz\x92]\x0b\r\xbb\x86\xe1\x80G%r\x01\xcf\x04\x83\x83\x8c\x10\x08\x9e\xdd\xdbQ\x86\xe4K\x88\xe3\x8aD1\xe5\xbeb\xc5\x08<6\xde6\x9e\xaaO\xcd\xc1Q\xb82\xd8\xcd\xe7Y\xa3\x1b\x98.d\\\xc7,\xb0\x0c!\x91IW\xc0\xcbm\xc3\x06\x1bI$c\x04\xe4RYM\xe7\xc0\xcf\xf6\x98.1,\x89\xbe\x01\x85\x1b]\x97i\xe4\xfc\xcb\x8d\xad\xcf\xdeS\xc0\xe8\x16d\xb8\xfbe\xb4\x91>a\x1b\x92h\xcb\x05\x18#!\xc7\xcaI`T\x007(\xc3\xb19!E\x00H\xd0\xa3\\$\xe4\xc9\xbd\x11\x90\x01#\x05\xc3\x10NW8\'\xe5\x18$ds\x8cd\xe4\x8e7G\x99\x9ag\x90;\xeeU`\xb8\x8cm\x03j\xe0\x03\x8c\x82y\xc9\xcb\x1eq\x80#\xbex\xd2\xceA%\xdf\xd8\xfc\xccD\x93\x82\xa0\xa3\xb9\n\xbbw\x02\xa5\xb7\x10\x00 \xe4\x900zU\x8a\x00\x8aa3F\x04\x12F\x8f\xbdI.\x85\x86\xdd\xc3p\xc0#\x92\xb9\x00\xf6$\x1c\x1c`\x8b\n\xad\xc3\xce\x0c\x9b\xdd\x15\x0821\\)$as\x80~c\x92\x06O\x19\xce\x06%\xa4\xa0\x08#v\x9e\xd1$\x92\x19 w@\xcd\x0c\x85K!#\x95;I\x19\x1d8$z\x13^\'\xe3\xfb\x1b{;\x86\x8d.d2K;J\x8a\xee\x19\x95r7\x05\xc8\xce\xdc\x9e\xf9\xc6\xec\x0c\r\xa0{l\x86_=T"\x18J1w.C\x06\xc8\xda\x02\xe3\x04\x11\xbb\'#\x18\x1c\x1c\xf1\xe3\xdf\x12\x1b.\xa3\xb0\x92\x808\x15\xe9R\xb4\xe5aD\x92b"W\xca\xab7\xca\x19\xb08\x1e\xa7\n=\xf0)\x8f+\x98\x14\xec\r\xe5\xa9\n\xa8\xaa\xa5\xb9\'\x93\xc6NOR}\x06p\x058\x0c\xf1@\x1e\xd1\xf0\xd2\xd4A\xe1\xf0\xc3\xf8\xd8\xb5l\xff\x00\xc2\t\xe1\x0f\xfa\x154?\xfc\x17C\xff\x00\xc4\xd5O\x00\x00<5\x00\x18\x15\xd5\xd0\x07\xc5>\x01\xb8\x99|Yan\xb2\xc8 y\xd5\xde0\xc7k2\x82\x14\x91\xd0\x90\x19\x80=\xb7\x1fZ\xfb\x03J\x89Ff\xcb\xeftT \xc8\xc5@\x04\x91\x85\xce\x01\xf9\x8eH\x19<g8\x18\xf8\xd7\xc1\x971\xda\xf8\xb7O\x96V\xda\x82Q\xcd}\x97\xa5I\xe6@\x8e\x08\xf9\x86x\x18\xc5\x00G\xe2+\xc9-,\x08\x12A\x1cn\xad\xcb\x86y\x1d\x97\x0ec\x8e%\xe5\xd8\xc6\xb2\x9e\x0eAPv\xb0\xc8\x1e-\xe2p\xebas+\xc6\xbb\xd5\x98\x85V,\x08\x04\xe3\xb6rF2\x008\'\x1c\xf7\xf7y-Ule\x85\x19\xc8b\xed\x99$g9bX\xf2\xc4\x9cd\xf0:\x01\x800\x00\x15\xe3\x9e&\xb3\xfd\xd5\xdca\xc1\xd8O\xcc\x07\\P\x07\x9f\xa0\x9bsD\xd9\x08\x81\n\xcc\x18\x12\xfe\xa0\x8cq\xd3\xf5\xe3\x1d\xac\xbc\xc5\xd24!F\xc1\xb4a\x00$d\x9eH\xeay\xeay\xc6\x07@)b[\x8d;P\x8aG3%\xcd\xac\xdb\x8a\xb1(T\xab\x82T\x81\x82:c<0\xe7\x04\x1ei\xd3H\x0c\xc6\xdc\xdc\xac\xe2\x01\xb62\x18\xe0&\xe2APpB\x93\xb8\xf4\x1dO|\xd0\x06\xb7\x87\xe7Qp\xf0\x83\x97.\xbf*\x82q\xd4\x82q\xd0|\xa7\x93\xc6x\xeb^\xf3do6X\x18D\x06\xd4\xa1\x13\xab\xe48\xe3\xe5d#\x83\x820T\x81\x9d\xd9\xdc6\xedo\x00\xd1#7\x13\\\xc0^\x10\x8d\x18\xde\x92\xa1q"\xe4\x06\\{\xa9=x\xfa\xf4>\xed\xe1y\xe4\x9fK_29\x17a\n\xac\xe4\x11 \xda\x0eF\t8\xc9#\x9c\x1c\x83\xc60H\x06\xcc3$\xe8]\x04\x80\x07d\xf9\xe3d9V*x`\x0e28=\x08\xc1\x19\x04\x1a}G9\x99c\x06\x08\xe3w\xde\xa0\x87r\xa3n\xe1\xb8\xe4\x03\xc8\\\x901\xc9\x00dg!g\x82\x1b\xabym\xee"I\xa0\x95\nI\x1c\x8a\x19]H\xc1\x04\x1e\x08#\xb5\x00\x10\xcc\xb3\xa1t\x12\x00\x1d\x90\xef\x8d\x90\xe5X\xa9\xe1\x808\xc8\xe0\xf4#\x04d\x10h\x95\xa6\x12B"H\xd9\x0b\xe2R\xceT\xaa\xed<\xa8\xc1\xdcwm\x18$pI\xcf\x18$\x11\xb46\xf1F\xf3I;\xa2\x05id\n\x19\xc8\x1fx\xed\x00d\xf5\xe0\x01\xe8\x05W\xbf\x82\xee\xe9>\xcfos%\x9a:\x12n\xa0d2\xc6\xc1\x97\x01U\xd1\x90\x827\x02ON0\x0er\xa0\x16!\x91\xa5B\xcf\x0c\x90\x90\xec\xbb\\\xa9$\x06 7\xcaH\xc1\x03#\xbe\x08\xc8\x07 $\xc2f@ \x924}\xeaIt,6\xee\x1b\x86\x01\x1c\x95\xc8\x07\xb1 \xe0\xe3\x05\x88-\xbf\xb4f+\x06.\x8cQ\x89&\xf2H\xdc\x99}\xab\xbf\x18l\x1d\xe7nN\xdd\xd9\xc0\xdc3=\x00A-\xa7\x9bymr.\'\x8d\xa1\xdc\n#\xfc\x92\xab\x0eC)\xe0\xe0\x85 \x8c0\xc6\x01\xc30i\xea\x0b\xc5\xf3b\x16\xc6)\xda;\x8d\xd1H\xf0\xc9\xe5\x98\x94\xa9\xf9\xb7\x06\x0c:\x00\n\xfc\xc0\xb0<\x0c\x90XM\xf6\x8d:\xdao\xb4\xc1u\xe6D\x8f\xe7\xdb\x8cG.@;\x90e\xbeS\xd4rx=OZ\x00\x9e\xa0\xb5\x87\xcb\xf3\xe4kh \x9aiY\xe40\x9c\xf9\x98\xf9U\x98\xe0e\x8a*g\xd3\x18\xc9\x00\x1a\x92\x06\x99\xad\xe2k\x88\xe3\x8er\x80\xc8\x91\xb9uV\xc7 1\x00\x91\x9e\xf8\x19\xf4\x15%\x00E\x00\x99m\xe2[\x89#\x92p\x80H\xf1\xa1Ef\xc7$)$\x81\x9e\xd98\xf55-G0\x99\x90\x08\x1e4}\xeaIt,6\xee\x1b\x86\x01\x1c\x95\xc8\x07\xb1 \xe0\xe3\x05 \x13-\xb4kq$rN\x10\t\x1e4(\xac\xd8\xe4\x85$\x903\xdb\'\x1e\xa6\x80\x15\xa7\x85n\x12\xdd\xa5A<\x88\xce\x91\x96\x1b\x99T\x80\xc4\x0e\xa4\x02\xca\t\xed\xb8z\xd4wW\x91Y\xf9\x1ejN\xdet\xab\ny0<\xb8c\xd0\xb6\xc0v\xaf\x1c\xb3aGr(O+\xfbFm\xbe\x7f\x9d\xe5G\xbbv\xff\x00/n_\x1bs\xf2n\xce\xec\xe3\xe6\xc6\xdd\xdcm\xab\x14\x00Ug\xfbg\xf6\x8c;<\x8f\xb0\xf9Ry\xbb\xb3\xe6y\x99M\x9b{m\xc7\x99\x9c\xf3\x9d\xb8\xefJ\xf7\x0eo\x16\xde(\xb7m\xc3L\xce\x19B\xa1\x0f\x82\xa7iWm\xca\x01\\\x8c\x03\x93\xfc!\xa3\x16\x0f\xf6\x8by\xda\xfa\xed\x9e\x17\x94\xe3r\x85\x91\x1c\x93\xb1\x94\x0c\x10\xbf.\xd6\xc6\xe1\xb7\xef\x1d\xcf\xb8\x02\xe5F\x0c\xdfhuh\xe3\x10\x04R\x8e\x1c\x96-\x93\xb8\x15\xc6\x00\x03n\x0eNrx\x18\xe5\xf4\xb4\x01\x1c"eB\'x\xdd\xf7\xb1\x05\x10\xa8\xdb\xb8\xed\x18$\xf2\x17\x00\x9e\xe4\x13\x81\x9c\x01\xa3sp\x92\x89\xa4\x08\xa8\xcab\x01v\xb1$a\x8f\x19\xc8\xc1\x03\x04\x0f\x98\xe4\x1e0O2\xdb[\xcb;\x89\nF\x85\xd8G\x1b;\x10\x06xU\x04\xb1\xf6\x00\x93\xda\xa4\xa0\x04\xac\x8f\xed\xeb\x7f\xed\x1f%\xe6\xf2\x02\xcb\xf6qo,\'\xce\x9d\xd8\xe1\x1d\x14\x1d\xc2?\x92q\x92\x9c\x88\xd9\xc1\x08\xbb\x9b^\x99\x1c\x10\xc2\xf3<QF\x8f3\xef\x95\x95@.\xdbB\xe5\xbdN\xd5Q\x93\xd8\x01\xda\x80\t!Y^\x17c 1>\xf5\xdb#(\'i_\x98\x03\x86\x18c\xc1\xc8\xce\x0fP\x08\x92\x8a\x8e8R)&u.L\xaf\xbd\xb7H\xcc\x01\xda\x17\xe5\x04\xe1F\x14p03\x93\xd4\x92@$\xa8\xa0\x91\xa6\xb7\x8aW\x86H\x1d\xd03E!R\xc8H\xfb\xa7i##\xa7\x04\x8fBid\x85%x\x9d\x8b\x83\x13\xef]\xb22\x82v\x95\xf9\x808a\x86<\x1c\x8c\xe0\xf5\x00\x88\xa5\xfbcJ\xcb\x17\x91\x1cc\xcb+#\xe5\xcb|\xc7\xccR\xbcc\xe5\x03\r\xb8\xf2\xdc\xae\x17\x0c\x00\xe8\x96eF\x13\xc9\x1b\xbe\xe6 \xa2\x14\x01w\x1d\xa3\x04\x9eB\xe0\x13\x9eH\'\x038\x1eS\xf1*8\x16\x19\x9ah\xbc\xe2Jl,\x0b\x14;\x86\x08\x03\xd3\x1d\xf8\x03\x9e1\x91\xea\xd3\\C\x0b\xc5\x1c\x92\xc6\x8f3\xec\x89Y\x80.\xdbKa}N\xd5c\x81\xd8\x13\xda\xbc\xab\xe2$R\xbd\xa0\xfb\\\xa2I\xe1\x0b\xb9\xa2\r\x1a3`\x83\x85\xdcp\xb9$\xed$\xe3\x8eI\x19\xa0\x0f7\x89L\x82L\x10\x0ch\x1d\x83\x1cd\x13\x8e3\xf7\x8e{\x0c\x909\xe9M\x12\x18\x80.w1b\x06\xd58\xeeFz\xe3\x8e\xfe\xbf\\S\xfc\xe7\x86\xf5f\xb3\xdf\x08Y7FL\xbf<`r\xa40\x03,\x0e9\xe3\xd7\xda\x92\x00\x1a\xec,\xc5\x92\xdf\xe5\xcb\xa2\x06#\x93\xbb\x03p\xc9\xc68\xe3\xeb\xe8\x01\xec\x9f\x0ee\x95t\x98\xa1\x8e=\xd9`\xf2\xb3\xeeP\xa8C`\xa9\xdaC6\xe5\x00\xaeF\x01\xc9\xec\x1b\xbf\xae_\xc1\x16\xed\x07\x87\xad\x83)RT6\x08\xc7\x04d\x1f\xca\xba\x8a\x00\xf8CE\xff\x00\x90\xdd\x97\xfdv_\xe7_ih\'\xfd\n/\xf7G\xf2\xaf\x8a\xf4\x87X\xf5\x8bGb\x02\x89T\x92~\xb5\xf6W\x87Qn\xad-\xe6\x9f\xc9\x9cD\xc2KS\xe5\xf3\x10)\xb4\x9c\x92~o\x99\xf9\x18\xe1\xb1\xeaH\x07Au\n\\[\x94r\xe0\x06W\xf9$d9R\x18r\x088\xc8\xe4t# \xe4\x12+\xc8\xf5\xa8\xcaG\xa8\xabH\xd2\x1c\xb9\xcb\x01\x90\t$\x0e\x00\xe0t\x1d\xf09\xc9\xe6\xbd\x85\x86c#\xda\xbc\x8f\xc4V&;\xbdIPD\xb72\xa1Q!\\\xf02T\x1e\x84\x80I\xe3=\xcf\xad\x00y\xac\x90\x95_-\xc8\xf9\x90\x1c\xc7 <0\xcfU<\x1c\x1f\xa8<pEKl\xf6\xac\xb1\xcd%\xb2\xb4\x91\xfc\xe9\xb9\x1b2\x02W\xf7o\xf3\x02\xab\x80O\x1c\xe7\x83\xd8\xab\xe4\xb7\x86\x19\x104\xad\xe5\xb4E\x83*\xa9m\xc0\x11\x82\xa1\xb8\x1b\xc62y\xdb\x86\xc7 TQ4j\xe4\xc8\xac\xcb\xb5\x86\x15\xb6\xf3\x83\x83\xd0\xf0\x0e\t\x1d\xc7\x19\x1dh\x03KE/\x06\xb7,\x0c\xa3*\xa5K+\xab)9 \xe0\x82A\xe9\xd4pr0M{\x97\x86\xaeR-\x0e\x07q!\x05\x950\x91\xb3\x9c\xb3\x05\x1c(\'\x19<\x9e\x80d\x9c\x00Mx&\x8d\xe6\xbd\xe4r\xc0\x92\x15\x003\x9d\x87\xe5R@\xcbzr@\xe7\xb9\x02\xbd\xbf\xc1\x17\x0f%\x83\xa3\x1f\xb8\xf8\x14\x01\xd1@.\xee,"[\x87{{\x95p$x\xd1\x17\xcc\xd8\xfc\x90\xa4\xb8Tp:d\xb0V\xea\x18qn\x98\xd0B\xd7\tp\xd1FgDdI\n\x8d\xca\xacA`\x0fP\tU$w\xda=*J\x00\xafo}os,\x91E\'\xef\xa2\xe5\xe2u*\xea72\x86*pv\x92\x8f\x86\xc6\x1b\x19\x04\x8ej\xc5W\x87\xca{\xcb\x99\x13\xcf\xf3\x17l.\x1fxN\x06\xe0T\x1f\x94\xfd\xfeYz\xe3\x04\xe5p\x0b\xe5\xddg#yS\xcc\xd1\xe2U\x8a\t<\xb7\x91\x94\x86\n\x0e\xe5\x1c\x90\x06\t\ns\x83\xc14\x01$\xd0\xa4\xf1\x84s \x01\xd5\xfeI\x19\x0eU\x83\x0eT\x83\x8c\x8eGB2\x0eA"\x88\x16e\xb7\x88\\H\x92N\x10\t\x1e4(\xac\xd8\xe4\x85$\x903\xdb\'\x1e\xa6\xa4\xa2\x80+\x87\xb9\xb8\xb3\x86H\xd3\xec\xb3>\xc7x\xe7P\xe5\x06Ad;[\x1b\xb1\x91\x90\xc4\x03\xcf\xcc\x06\t|\xf7\x11\xe9\xd7/f\x9b\xee\x96&0\xae\xd0\xdb\x9f\x07h\xc1e\x07\x9cpY~\xa3\xadI\x1cl\x8f+4\xd2H\x1d\xf7*\xb0\\F6\x81\xb5p\x01\xc6A<\xe4\xe5\x8f8\xc0\x11\xcd$w\x1fh\xb2\x8a\xf3\xc9\xba\x11\x02LEL\x91\x06\xdc\x15\xf6\xb0#\xaa\xb62\x08%O\x07\x06\x80$\x9e\x05\xb9\xb7\x96\x072\x04\x91\n1\x8eFF\x00\x8cp\xcaAS\xee\x08#\xb5\x13,\xcd\x18\x10I\x1a>\xf5$\xba\x16\x1bw\r\xc3\x00\x8eJ\xe4\x03\xd8\x90pq\x82A\n\xdb[\xc5\x02\x19\nF\x81\x14\xc9#;\x10\x069f$\xb1\xf7$\x93\xde\xa4\xa0\n\xf0\xa5\xe2\xfd\x9f\xce\x9e\x07\xdb\x11\x13\xec\x84\xae\xf9>\\2\xe5\x8e\xd5\xe1\xbeS\xb8\xf2>n\x0eKK\x0b=>#\x15\x95\xa4\x16\xd1\x9d\xb9Hc\x08\x0e\xd5\x08\xbc\x0fEUQ\xe8\x14\x0e\x82\x92\xc5\xee\x1e\xce?\xb5\xa6\xdb\x85\xcaHB\x85\x0eA#z\xa8f\xc2\xb67\x00X\x90\x08\x07\x9c\xd5\x8a\x00Z\xaff\x97\t\x03\x0b\xa7\xdf\'\x9b!\x07po\x90\xbb\x14\x19\n\xbd\x17h\xc68\xe8K\x1f\x98\xd7\xb2\x96\xe6mF\xf6Gi\xcd\xaf\xca\xb0+\xc0"E*Y\\\x0c\x9d\xec\xdb\x81;\x88\x08T\xc7\xb7?3\x1bs,\xcc\x80A"#\xefRK\xa1a\xb7p\xdc0\x08\xe4\xae@=\x89\x07\x07\x18 \n#ap\xf2\x99\xa4(\xc8\xaa"!v\xa9\x04\xe5\x87\x19\xc9\xc8\x07$\x8f\x94`\x0es\x1b\xc3\xf6\xb8\xee\xad\xafm\xa0\x92\xd6L\xc6\x11\x8f\x98%\x8c\xa8\xdd\xbdH\xc0\xc9,6\xf2\x08\x00\xe7\x9c\t\t\x9b\xed\x08\xa28\xcc\x05\x18\xbb\x97!\x83dm\x01q\x82\x08\xdd\x93\x91\x8c\x0e\x0ex\x89\xaf\xec\xd3\xcc\xddw\x02\xf9R\xa4\x12f@6H\xfbv\xa1\xe7\x86;\xd3\x03\xa9\xdc\xbe\xa2\x80,UH5(n\xaf\xee- Y$6\xcec\xb8\x90\x00\x16)6F\xe1\x0eNI+*\xb0*\x08\xe0\x82A\xc07(\xa0\x035\x1cm1yD\x91\xc6\xa8\x1f\x11\x15r\xc5\x97h\xe5\x86\x06\xd3\xbbp\xc0\'\x80\x0ey\xc0. \x86\xea\xde[{\x88\x92h%B\x92G"\x86WR0A\x07\x82\x08\xe3\x15]\xae\x05\xcd\xba[\xb4\xb2X^\\\xdb\xb3\xa4e\xa33\xc5\xc0\x0c@\xf9\x94\x94.\xa0\x9f\x99rGPy\x00\xb7E\x14P\x05\x7f\xb7[\x98<\xe8\xe4\xf3\xa3\xf3|\x92`S.\x1f\x7f\x96A\xdb\x9ca\xb2\x1b\xfb\xb89\xc0\x07\x16(\xaaw\x970\xad\xc5\xad\x91\xba\x92\x1b\x9b\x87\xdd\x1a\xc4\xa1\x99\x95\x08g\xceA\x011\x85,q\x8d\xe0\x02\x19\x96\x80.TsA\r\xca\x04\x9e(\xe5@\xea\xe1]C\x00\xca\xc1\x94\xf3\xdc0\x04\x1e\xc4\x03D\xc6e@`\x8e7}\xea\x08w*6\xee\x1b\x8e@<\x85\xc9\x03\xb9\x00dg"9\x9e3ym\x0f\xdb<\xa9\xbeiD\x00\xaefE\x1b[ \x82v\x82\xe8I\\`\xed\xc9\xc1\xc1\x00w\x94\xc2\xe5\xe53HQ\x91TDB\xedR\t\xcb\x0e3\x93\x90\x0eI\x1f(\xc0\x1c\xe7\xcf> Z\xac\x96\xb7A\x9b\x8d\xbb\x86+\xbcq\x0c\x9a\xc20{F\x9e\xde\xdd\x83\xa1@gE\x91\x86\xd2\x1b9T&&\xc8\xc7\xccTr6s\xe7\xff\x00\x11\xe4\x91w\x04\nP\xae\x1c\x96\xc6\x06\x0f#\x8eNq\xc7\x1dO\xa6\x08\x07\x92\xb4SAq$s,\x8b"1W\x8d\xd7\x0628#\x18\xcfPz\xf7\xa9\x1a9\x9e\xd6I!\xdb\xb9q\xc3\xe7\x07\xdb\x8e\x9f^\xde\x87\xa5GlbY\x83M\x1c\xa5\x0b\xfc\xe8$\xc1 q\xc1\xe4.@\xe3\x8e\xf9#9\xab\x96\xa1\xdbH\xbcuO\x9bx\x18c\x8e3\x82x\xcfnG\xf4\xa0\x0fh\xf8p\xd3\x1f\r\xc5\xe7c9 \x00\xdb\xb8\xcf\x1d\x87lq\xdb\xdf\xadvu\xc8|?\xb5\x8a=\x02+\x81\x12\t\xa4EF\x90(\xdc\xca\xb9 \x13\xd4\x80Y\xb0;d\xfa\xd7]\x93@\x1f\x02\xc5\xfe\xb9?\xde\x15\xf6\'\x84d\xf2<?gv\xf2\xcf\xe5ChK\xc3\x14{\xf7\xf0\x0eB\x85.X`\x80\x17\xae\xe3\xc18\xc7\xc7q\x7f\xaeO\xf7\x85}\x8f\xe0g\r\xa0Y`\xe4yK\xd3\xe9@\x1dZ\\I0\x89\xad\xe2\xcc~k\xc7)\x984l\xa1w\x0c\xaa\x95\xf9\xbe`\x00\xe8\n\x9d\xc0\x91\x8c\xf9\xf7\x8a-\xa4:\xdb\xe0`I\xf2\xab1\xda\xb9\xe3\xb9\xe3\xb8\xfaf\xbd\x02Kn.&\xb5X"\xbe\x92!\x18\x9d\xe2\xdd\xf7w\x14\x0c\x01\x05\x94\x16c\xb7#\xa9\xc1\x19\xcdy\xef\x8c\xac\xad\xc6\xb5q,\xb6q\x19\xee`\xf2V\xe8eeT\xceJ\xab)\x04\x0c\xff\x00\x88\xc1\xa0\x0f%\x93\xfdt\x9c\xff\x00\x11\xa7Z\xc2\xd7d\x15\xcaD\x02\xb4\x92:6"R@\xdc\xc0\x02@\xcb\x0e\xd9\xe7\x1dj{\xb8\xa1\xb7\x96y$\x89\xf6\xc4\n\xc8\x16e\xdc\xcew\x95*\xb8\xce\xd1\x81\x90\x01\xe9\xd4n\x18\xa7\x9cP\x05\xcd&f\xfb|b\x19\x88G\x7f.@\x8d\xc3\x00\xc0\xe0\xe3\xae\x19G\x1e\xa3\xda\xbd\x9f\xc0\x88\x16MAE\xc8\x90\xc7(\x89\xe2VVX\xd8s\x9c\x8erC/r0\x14\x8cd\x93\xe2Qy\xf0\xc9ov\xaaD^n\x03\xb2\x9c1\x00d\x03\xea7)=z\xfb\xd7\xb1\xf8\x02f\x13\xddC\xe7\xf9\x9b\x98\xc8C\x0f\x99s\xcf`\x069\xc0\xef\xf2\xf3\xea@=\x02\x8a*2&\xfbB2\xc9\x18\x80#\x07B\x84\xb1l\x8d\xa46p\x00\x1b\xb20s\x91\xc8\xc7 \x00i\xbe\xd0\xea\xc8\x82\x00\x8aQ\xc3\x92\xc5\xb2w\x02\xb8\xc0\x00m\xc1\xc9\xceO\x03\x1c\x934\xca\x80\xc1\x1cn\xfb\xd4\x10\xeeTm\xdc7\x1c\x80y\x0b\x92\x07r\x00\xc8\xceA<\x10\xdd[\xcboq\x14sA*\x14\x929\x142\xba\x91\x82\x08<\x10G\x18\xa2C0x\x84Q\xc6\xc8_\x12\x96r\xa5Wi\xe5F\x0e\xe3\xbbh\xc1#\x82Nx\xc1\x00\x92\xa3S7\xda\x1dY#\x10\x04R\x8e\x1c\x96-\x93\xb8\x15\xc6\x00\x03n\x0eNrx\x18\xe6;[w\xb7\xf3\xd5\xa5\xdf\x1bJ\xd2F\tb\xca\x1b\x92\x0b3\x1c\xfc\xc5\x88\xc6\x00R\xaa\x06\x17\x9a\xefm\x1d\xb5\xd47\xd3\xcd<\xf2G\xe6B\xa7\xc8W`&\x95\x08\x19D\xdc\x15v\xa8\xe0\x81\xb4n|\x95\xdc\x00.L\xb34c\xc8\x91\x11\xf7\xa9%\xd0\xb0\xdb\xb8n\x18\x04rW \x1c\xf0H88\xc1\x96\xa9\xd8[Cj\xf7\x8b\x15\xac\x90\x99.\x0c\x8f#\xb0c;2\xa9/\x9c\x93\x81\xc2\x00\xd8\xc0@\x00\n\x16\xac\t\x95\xae^\x00\x1fz"\xb9&6\x0b\x86$\x0c60O\xcar\x01\xc8\xe38\xc8\xc8\x04\x94RT\x17w?e\xf2\x1d\x9a\x04\x85\xa5X\xe4y\xa5\xd9\xb7w\xca\xbbx;\x98\xb9E\x0b\x91\xf7\xba\xe4\x00@\x0c[\xe9\xf0b8<\xb8\xda\\\x95\x82\x12~y\x1f\x96!Gvb\xcc\xde\xe5\x89\xc6MI<m5\xbc\x91G4\x90;\xa1U\x960\xa5\x90\x91\xf7\x86\xe0FG^A\x1e\xa0\xd2\x0bxV\xe1\xee\x16(\xc4\xee\x8a\x8f Q\xb9\x95I*\t\xea@,\xc4\x0e\xdb\x8f\xad,3\xc3r\x86H%\x8eT\x0c\xc8Y\x180\x0c\xacU\x87\x1d\xc3\x02\x08\xecA\x14\x00\t\xd5\xae\x1e\x00\x1fz"\xb9&6\nC\x12\x06\x1b\x18\'\xe59\x00\xe4q\x9cde\xf5\x05\xcd\xbb\xdc\xee\x89\xa5\xdbk$O\x1c\x8a\x85\x92BN\x00+"\xb0)\x81\xbb\xa0\xceH \x8cr%\xa7\x97x\xd7\tq8W\xc9xK\xeeFb\x10\x023\x92\xb8\t\xc0R\x17\xe6bA\'4\x00Cn\xff\x00\xe8\xf2\xdc\xcb\xe6]G\x11\x8d\xda"\xd1\xc6\xc4\xed,|\xbd\xc4u^2IPH\x07\x93\x97\xb0\x98\xdc#,\x88 \x08\xc1\xd0\xa1,[#i\r\x9c\x00\x06\xec\x8c\x1c\xe4r1\xcc\xb5\x14+2\xa1\x13\xc9\x1b\xbe\xf6 \xa2\x15\x1bw\x1d\xa3\x04\x9eB\xe0\x13\xdc\x82p3\x80\x00\xb1\x99\x8c\x92\x89cE@\xf8\x88\xab\x96,\xbbG,06\x9d\xdb\x86\x01<\x00s\xce\x04^N\xcdG\xcfKh?y\x16\xc9g\xce$;NQq\x8f\x99~y\x0f$`\x9e\x01\xdcH\xb3U\xee,\xe3\x9eH\xe6\x07\xca\xb8\x8f\xe5Y\xd1\x14\xb8B\xca\xce\x80\xb08V\xd8\xa0\xe3\xd3\x82\x08\x04\x00>\x19\x1eT,\xf0\xc9\t\x0e\xcb\xb5\xca\x92@b\x03|\xa4\x8c\x102;\xe0\x8c\x80r\x03\x16)"\xbcc\x1a\xe6\x19\xb3$\xae\xf3\xb1*\xe0"\xa8T \x80\xa4\x02N\x08\xc1\x19\xc1,H\x90L\xadp\xf0\x00\xfb\xd1\x15\xc91\xb0R\x18\x900\xd8\xc1?)\xc8\x07#\x8c\xe3#2P\x02QQ\x89\x95\xae^\x00$\xde\x88\xaeI\x8d\x82\xe1\x89\x03\r\x8c\x13\xf2\x9c\x80r8\xce23\x1b\xdc<W\x8b\x14\x91~\xe6L,N\x81\x98\xef\xc3\x96\x0c\x02\xe1\x14\x05\x18by\'\x1c\x1cn\x00\xb1Q\xc2&X\xc8\x9eH\xdd\xf7\xb1\x05\x10\xa8\xdb\xb8\xed\x18$\xf2\x17\x00\x9c\xf2A8\x19\xc0\x92\xa2\x8ee\x96IQC\x83\x13\xecm\xd1\xb2\x82v\x86\xf9I\x18a\x86\x1c\x8c\x8c\xe4u\x04\x00\x08\xfe\xd8\x87Q\xfb\x12\x8d\xd2,^l\x85]\x7fv\t\xc2\xee\\\xee\xf9\xb0\xf88\xc7\xee\xdb\x90q\x90\x0b}F\xce\x19$\x87\xcc\x85\xf6L\x89<%H \x87RQ\x80*\xc0\x80y\x00\x82;\x11R4\x8c.\x12!\x0c\x85\x19\x19\x8c\xa0\xae\xd5 \x8c)\xe799$`\x11\xf2\x9c\x91\xc6d\xa0\n\xf6\x96\x89e\x11\x86\x13\xb6\xddv\x88`TUH\x10(P\x88\x14\x0c/\x19\xe7=O8\xc0\x1eo\xe3hn\xe6\xd2`\x92\xe8\xacWRB\x86U1\x81\x86\xc0.6\x87`;\x8e\x19\x80\xf5n\xfe\x9ek\xce\xbe$\xcc\xb0\xd9\x19[8U$\xed\x04\x9f\xc8rh\x03\xc7\x94mf,\xc7\x04\xf0\x0e8\xf6\x1f\xcf\xf1\xad\xcd\x1dd[\x1b\xa9\x07\x94UN\xf5\r\xb5\xfee\xc1\xf9\x90\xe7\x8eGQ\x83\xc8\xe7\x06\xb1\xd8\xa4\xeb\x11\xd8\xa9\xe5\x83\xca\xb3|\xe7\'\x96\xe7\x1d\x0e1\xd3\x81\xc6y\xae\x93H\xb4\x84xr\xfe\xe69\x92G)\xb6ERs\x19\xfe\xeb{\xe3\x07\xe8\xc2\x80=C\xe1\xd3\xca\xde\x16\x80J\xd9\xc7N*\xff\x00\xfc#\xda\xa7\xfd\x0eZ\xe7\xfd\xf9\xb2\xff\x00\xe4z\xa3\xf0\xf6\xd8\xc7\xe1\xabi\xdc2\xcb$J\xae\xa2Vd\x18\xc9\xe0\x1e3\x96<\x80\t\xe3=\x06:\xea\x00\xf8\n\xbe\xb7\xf8^\x04~\x16\xb3\x8dfY\xd5\x01Q*n\xda\xe0\x1e\xa3p\x07\x07\xaf \x1fa_&G"\xa2L\xad\x0cr\x17M\xaa\xcc[1\x9d\xc0\xee\\\x103\x80G9\x18c\xc6pG\xd4\x9f\x0c\xee\xfe\xc9\xe0\x9d6O"y\xb2\xd1\xc5\xb6\x14\xdc\xc3{\x85\xdcG\xf7Wv\xe2{\x00Oj\x00\xf4KE\xbcXe\xb9\x11bK\x99c\x97\xec\xf3\xc8A\x81J\xc6\xae\xa4\x86u\xdc0\xc7\t\x85\'\x8e\xa4\xb9\xe5|n\x01\xbd\xb5\xfck\xb9_\xba+\x85\xf1\xe4*\xf2\xda<\x91\x07\x8d\\\x1f\x98d\x02\x08 \xfdA\x00\xfe\x14\x01\xe4\xda\xbb\xdc}\xbe\xe8\x88\x94\xc2\x81ag\xf2W\x8c\xb1q\x96\xc7\xde;O9\xce\x01\x1d8\xaa;\xa5{L\x193\x0crq\x19\x90p\xcc9!s\x9eB\x0c\x90;.{U\xad~\x16\x8b\\\x99\xd8\x9c8\x05G\x18\xfa\xff\x00\x9fJ\xa0\xf2\x97X\xd4\x85\x01\x17h\xda\x80\x122O$u<\xf5<\xe3\x03\xa0\x14\x00\xbb\x97j\xa8\x8c\x86\r\x92\xfb\xf2\x08\xe3\x03n8\xc7<\xe4\xe7=\x069\xf5o\x87r\x93\xad\xdd(1\xb2\x88\x90\x1d\xaf\x96\r\x8c\x90F8\xe0\xa9\xeb\xcezz\xf9+:\xc6\xa5\xdd\x82\xaa\xf2I8\x00\n\xf5o\x03\xde\x91\xa9@\xd1\xa6a\x96\x15\x01\x9b\x86\xdd\xfcC\x1e\x83\x8c\x1c\xf3\xe8(\x03\xd4\xa4\x8d\x9d\xe1e\x99\xe3\x08\xfb\x99T.$\x1bH\xda\xd9\x04\xe3$\x1e0r\xa3\x9cd\x129\x96W\x99\x14H\x0cO\xb1\xb7F\xca\t\xda\x1b\xe5$a\x86\x18r23\x91\xd4\x10\x1a\xe2fhLo\x1a\xa0|\xca\x19\x0b\x16]\xa7\x859\x1bN\xed\xa7$\x1e\x01\x18\xe7!\x976V\xf7\x9b\x85\xcc~tm\x13\xc2\xf1;\x13\x1b\xa3\xe3pd\xfb\xad\xd3\x1c\x83\x80H\x1c\x13\x90\x06Gm2\xd9Kf\xb2In\x88\x9eU\xbc\xeb1\x9a]\xbb\x00\x0e\xc6E?8l\xfd\xed\xf9\xc0$\x9c\x90,@fkx\x9a\xe28\xe3\x9c\xa02$n]U\xb1\xc8\x0c@$g\xbe\x06}\x05\x0b\x04+p\xf7\x0b\x14bwEG\x90(\xdc\xca\xa4\x95\x04\xf5 \x16b\x07m\xc7\xd6\xa4\xa0\n\xf7\xcfy\x1d\x9c\x8da\x04\x13\xdd\x0cl\x8ey\x8cH\xdc\x8c\xe5\x82\xb1\x1cg\xf8O\xe1\xd6\xa3]\x966\xef}\xa8]"\xba[\xaf\xdaf.\xd1\xc0\x02\x02Y\xc23\x10\x83\x96$\xe78\xc0$\xed\x15r\xab\xbby\xb7\x8b\x12K<m\x0e%p#\xf9%V\x0e\xa1K\x15\xc1\xc1\x1b\x88R\x18as\xc3a\x80,\xd1IP-\xaf\xfa\xb3-\xc4\xf2\xb4R\xbc\xaa\xc5\xf6}\xed\xc0)\t\x80\xca\x03`\x06\x07\xa2\x93\x96\x1b\xa8\x02\xc5\x15\x1c0$\x11\x94C!\x05\xd9\xf2\xf23\x9c\xb3\x16<\xb1\'\x19<\x0e\x80`\x0c\x00\x05F\x97>\\V\xabz\xd0Au>\x10D\xb2\xee\x06M\xa5\x99P\x90\x0b`+\x1e\x80\xe1I\xc0\xa0\x0b\x14\x94\xb5\x1c\x92<o\n\xac/ w\xda\xcc\xa5q\x18\xdaN\xe6\xc9\x07\x19\x00q\x93\x96\x1cc$\x00IQ\xc7\n\xc5$\xce\xa6Be}\xed\xbaF`\x0e\xd0\xbf(\'\n0\xa3\x81\x81\x9c\x9e\xa4\x92,*\xb7\x0f82otT \xc8\xc5@RH\xc2\xe7\x00\xfc\xc7$\x0c\x9e3\x9c\x0cI@\x14\xf4\xf9\xacnVi\xec\x84a\xe4u{\x80#\xd9 s\x1a\x11\xe6)\x01\x95\xf6y|0\x04\r\xbe\xd5b\x19\x96t.\x82@\x03\xb2|\xf1\xb2\x1c\xab\x15<0\x07\x19\x1c\x1e\x84`\x8c\x82\r\x00M\xf6\x87f\x923\x01E\x08\x81\x08`\xd9;\x89l\xe0\x826\xe0`c\x07\x93\x9e$\xa0\n\xd6\xe6\xdeyd\xba\xb7\x9f\xcd\xdd\xfb\x96)1d\x066`@\\\xed\x0c\x18\xb08\x19\xe3\x07\xee\x8cY\xaa\xff\x00h\x97\xfbG\xec\xdfb\x9f\xc9\xf2\xbc\xcf\xb5e<\xbd\xd9\xc6\xccn\xdf\xbb\x1c\xfd\xdd\xb8\xef\x9e*\x99v\xb7\xb7\xd3\xe1\x89\xe4\xb7u\xb8\xf2#\x86\xee\xe5K\\\xa2\x86\x07.w\xb3\x9d\x8a\xd2\x8eC\x1d\x83q_\x9a\x805*\x9d\xce\xabcgq\x1d\xbc\xf71\xa4\xae\xf1\xa6\xde\xbbL\x85\x84{\xb1\xf7C22\xa9l\x02\xdf(\xc9 T\xe6\x15k\x84\x9c\x997\xa22\x00$`\xb8b\t\xca\xe7\x04\xfc\xa3\x04\x8c\x8eq\x8c\x9c\xb2\x15\xb8K\xcb\x9d\xef\xbe\xdd\xf6\xbcD\xb0\xca\x1ca\x90(Q\x85\xe00%\x98\x92\xec8\x00P\x05\'\xb66\x976\x91\xd9\xc9vdTE+,\xd2<o\n\x1d\xa7{\xb2\xb8\x0f\x89w\x0eU\xe4(\x01b\xaaqs\xed\x12\xf9\xfe_\xd8\xa7\xdb\xe6\xf9~fSn\xdd\x9b\xb7\xfd\xec\xed\xcf\xc9\xd3v\xee\xdb~j\x1e\x1b\x96\xd4a\x99n\xb6\xda\xa4R$\x96\xfeX>c\x92\x85_wQ\xb4+\x8cw\xdf\xec(\xb9\x95\xed\xf7\\\xb3f\xd6(\x9d\xa4\x8d i$b0A]\xa7\'\x007\xca\x14\x96$c\x18\xc1\x00\xb0i)\x8do\x0b\\%\xc3D\x86x\xd1\x91$*7*\xb1\x05\x80=@%T\x91\xdfh\xf4\xa4\x8e\x15\x8aI]K\x93+\xefm\xd23\x00v\x85\xf9A8Q\x85\x1c\x0c\x0c\xe4\xf5$\x90\x00\x19\xbe\xd0\xea\xd1\xc6 \x08\xa5\x1c9,[\'p+\x8c\x00\x06\xdc\x1c\x9c\xe4\xf01\xccm\x0f\xda\xbc\xc8\xef-\xa0xVTxA;\xf7m\xda\xca\xc4\x106\xb0pq\x8c\xfd\xd59\xc9\xc0\xb1Q\x917\xda\x11\x96H\xc4\x01\x18:\x14%\x8bdm!\xb3\x80\x00\xdd\x91\x83\x9c\x8eF9\x00.![\x9by`s I\x10\xa3\x18\xe4d`\x08\xc7\x0c\xa4\x15>\xe0\x82;W\x9e|I\x82G\xb1.1\x80;\xd7wj\x93;\xab\xdd\xac\x9fh\x81<\xa3"\x12\x91M\xb9Q\x99\xd60\xed\x81\xb8`n\xf9\x97\x0c\x01\xc1\xcbr\x1f\x10c\x95ld\x99\xbc\xa7\x80G\xb5c*A\r\xf3e\x8b\x06\xe7\xaa\xe0``\xa9\xceA\xc0\x00\xf1xQ^)\x18\xc8\x88Ur\x15\x81\xcb\x1c\x81\x81\x81\xd7\x9c\xf3\x81\x80{\xe0\x1e\xc7C\xb8K\xcf\x0e\xea\x8aa\x8a\xda\xce\x15w\x95\xf7\xb2\x88\xf1\x82\x00\xdd\x9c\xae\t\xcb\x16\xe3\x1d\xf9\xc7\x1f \x13,\xd2\xc4\xd0\xc0\xe5\xfeH@b\x179<g\xaa\x8c\x01\xcbg\x91\xd7\x92;\xdf\x04G\x9f\x0c\xeaY\xe7\'\x14\x01\xdb\xf8/Q\x8a/\r\x03v\xe2\x08\xeda\xf3d\x96E)\x1a\xa7\xcd\xc9s\xf2\xf1\xb4\x93\xcf\x03\x04\xe0\x11\x9e\xbe\xb1<-m\xf6]\x12\x14\xa8\xff\x00\xe1\x1e\xd4\xff\x00\xe8r\xd7?\xef\xcd\x97\xff\x00#\xd0\x07\xc4\x93\xc6\xb0\xdcK\x12M\x1c\xe8\x8eUe\x8c0W\x00\xfd\xe1\xb8\x03\x83\xd7\x90\x0f\xa8\x15\xf4\x7f\xc1inf\xf0\x84A&P\xcb0\xe6E.6\x86\x1b\x80\x00\x8eH\xc8\x07\xb1 \xe0\xe3\x07\xe7\x88#\xb9\xd6\xb5\x98\xa2y\xa4\x96\xee\xf6\xe0+K yY\x9d\xdb\xef\x1d\xa1\x9d\x89\'<\x02O`M}\t\xf0b+\xab}\x0b\xec\xd2#[\xcbot\xf1\xdc\xc5<,\x1c\x10\x08*2F\xd2\x1b\x198=\x08\xc6y\x00\x1e\xd5\x1f\xdd\x15\xc9x\xce\xdd \xd2UP\xc8A\x99\x9c\xef\x91\x9c\xe5\x98\xb1\xe4\x92q\x93\xc0\xe8\x06\x00\xc0\x00W[\x17\xdd\x15\xcc\xf8\xbe\xd9\xbf\xb2\xe6v\x9aI\x03J\x1dQ\x80\xc4c\x00ap\x07\x19\x04\xf2I\xcb\x1eq\x80\x00<s\xc6FD\xd4-\xdet \xfd\x98\x1c\x82Y\x99y\xc1#$\x8e0\x00\xc0\xe0\x0e9\xc9\xc3\x8d\xd5\x18\x96\x8dd\x1bH\xc3\x13\x80H \x1e\x08\xe4u\x1d\xb29\xc8\xe2\xb6\xbcV\xa7\xfbB\'\x00\xec)\x80k\x9f\xf9\x1d\xb9\\\x98\xdb\x82W\xa1\xc7Q\xf8\x1e\xa3\xdcP\x05\xb6{=\x97 A8fa\xf6rf\x18\x8cg\x90\xff\x00/\xceq\x81\x91\xb7\x9eq\xda\xbd\x0f\xe1\xf4\xa2hl\x8a\x14dGq\xb9@\xcez\x11\x9f\xc3\xa7n}\xeb\xcd\x02\xae\xc9\xa5y\xb6\x84\x8fp\x8fi%\x8ez.\x019\xc1\xcf<ax\xe7\x83\xe8>\n\xd5|\xb4\xb6-\xe4*\xa4\xa9\x13\xb30\x8c\x00F\xd5\xc0\x03\x0c\xc5\xb6\x8e\xc4\x96\'$\xf0@=\x96Y\x1e\x1bY$Hd\x9d\xd1\x0b,Q\x95\x0c\xe4\x0f\xba7\x102zr@\xf5"\x99oagi,\x92\xdbZA\x0c\x92\xff\x00\xacx\xe3\n_\xe6g\xe4\x8e\xbf3\xbb}]\x8fRi\xd6\x9ew\x91\xfb\xf7Gr\xccAD*\x02\xe4\xed\x18$\xf2\x17\x00\x9e\xe4\x13\x81\x9c\x07\x08Un\x1epd\xde\xe8\xa8A\x91\x8a\xe0\x12F\x178\x07\xe69 d\xf1\x9c\xe0`\x02JZ\xad\xfb\xd9\xe7\xff\x00\x96\xf6\xeb\x04\xbf\xec\x15\xb8]\x9f\x89\n\x0b\x7f\xb2\xdb\x93\xfb\xbfyQ\xee"\x8a\xd69\x93\xed\x136\x12iaP\x88\xa7i%\xf6\xb3\x12\x14\x91\x80\x01b7\x0e\xd9 \x00x~\xcf\x15\xd4\xb6V\xd0\x1b\xa9s!\x0c|\xb1,\x81B\xae\xf6\x00\x9e\x8a\xab\xbb\x04\x80\x07\x07\x18\xa8\xec"\xb4\xb1O\xec\xdb{\x87\x91\xe0@\xe5\'\xb9y\xe5\n\xec\xd8,\xce\xc5\xc8$0\x04\x9f\xe1 t\xab\x95\x1a\xc8\xc6\xe1\xe20\xc8\x11QXJJ\xedbI\xca\x8es\x91\x80N@\x1f0\xc1<\xe0\x01g\x8d\xa6\xb6\x96$\x99\xe0wB\xab,aK!#\xef\r\xc0\x8c\x8e\xbc\x82=A\xa2{\x88mm\xe5\xb8\xb8\x9a8`\x89\x0b\xc9,\x8c\x15Q@\xc9$\x9e\x00\x03\x9c\xd2\x1bxZ\xe1.\x1a(\xcc\xf1\xa3"HTnUb\x0b\x00z\x80J\xaeG}\xa3\xd2\xa5\xa0\x08\xe4\x91\xd1\xe2U\x86I\x03\xbe\xd6e+\x88\xc6\xd2w6H8\xc8\x03\x8c\x9c\xb0\xe3\x19"\xbc\xb6s\xc9$\xec\xba\x9d\xdcbDuEE\x8b\x11\x12\xaa\x03.P\x9c\xa9RF\xec\x8c\xbbd\x11\xb4-\xca\x89`\x85n\x1e\xe1b\x8cN\xe8\xa8\xf2\x85\x1b\x99T\x92\xa0\x9e\xa4\x02\xcc@\xed\xb8\xfa\xd0\x04\x95R\xf27\xbd\x8e{\x16\xb7\x8c\xc0\xe8\x82F\xb8\x8dd\x8adf"H\xf6\xee\x07;A\x19#\x1f:\x9f\x9b\x05j\xe5V\xfb\x1c^\x7f\x9d\xbe}\xdeo\x9d\x8f=\xf6\xee\xd9\xb3\x1bs\x8d\xb8\xe7n6\xee\xf9\xb1\xbb\x9a\x00\xb1E2F\x98<"$\x8d\x90\xbe%,\xe5J\xae\xd3\xca\x8c\x1d\xc7v\xd1\x82G\x04\x9c\xf1\x83%\x00R\x81.c\x11H\xd0\xa0\x96w\x0fv\xbfjwX\x8f\x97\x83\xe5\xe5y\x1b\x95F0\x80\xe5\x9b\xaeAV\xd3-\xd8\xa12]\xfc\x8e\xce1w(\xe5\xa4\x12\x1c\xfc\xdc\x8d\xca\x00\x07\x80\xb9Q\x85%M\x88Ve\x8c\x89\xe4G}\xecAD*6\xee;F\t<\x85\xc0\'<\x90N\x06p\x1f@\x11\xad\xbc+p\xf7\x02$\x13\xc8\x8a\x8f Q\xb9\x95I*\t\xea@,\xc4\x0e\xdb\x8f\xadKIL\x9dfkyV\xdeH\xe3\x9c\xa1\x11\xbc\x88]U\xb1\xc1*\x08$g\xb6F}E\x00IEE:\xcc\xd6\xf2\xad\xbc\x91\xc79B#y\x10\xba\xabc\x82T\x10H\xcfl\x8c\xfa\x8aXch\xa3*\xf3I1.\xcd\xb9\xc2\x82\x01bB\xfc\xa0\x0c\x00p;\xe0\x0c\x92rH\x02H\xd3\t!\x11G\x1b!|JY\xca\x95]\xa7\x95\x18;\x8e\xed\xa3\x04\x8e\t9\xe3\x061af\xb6p\xd9\x8bH\x05\xac;<\xa8Dcd{\x08)\xb5z\r\xa5T\x8ct\xc0\xc7J\xb1U\xda\xf1\x0f\x98-\xc7\xdad\x8aT\x8aX\xe1u-\x19m\xa7\xe6\xc9\x18\xc2\xb8r:\xed\xe8\t \x10\t!\x8d\xa2B\xaf4\x93\x12\xec\xdb\x9c( \x16$/\xca\x00\xc0\x07\x03\xbe\x00\xc9\'$\xc6\xf6Q=\xe2\xdd\x16\x9cH\xb8\xc2\x89\xdc\'\x01\xc0\xca\x03\xb4\xfd\xf6\xceG8RyU\xc5\x8a(\x02\x96\x96o\x8d\x82\xff\x00i\x18\xcd\xd8wWh\xe2\xf2\x95\x80r\x15\x82\xef|\x02\xa0\x1eX\x9ey\n~Qr\xaaZ\xdcMq\x7f|\x8d\x1c\x91\xc1n\xe9\x12y\x90\x95\xf3\x1b`vum\xd8d\xc3\xaa\xf4\x18d~Oj\xfev\xae\xf7\xb6\xd2Gm\x1a\xdaKn\xad$\x13\xa8W\x86M\xe9\xb84\x8a\xec\t\xd8\xcd\x85T \x949\x91A\x19\x00\xd3\xae#\xc7B[\xef\x0e\xbd\xdd\xa4\xf0\\X\xc9\x12<-\x10\xceA\xc9,\x1c\x1c2\x90W\x18\x1d\x89\xc9\xc8\xc7h-\xe1[\x87\xb8X\xa3\x13\xba*<\x81F\xe6U$\xa8\'\xa9\x00\xb3\x10;n>\xb5\x89\xe2\xe0\x1bA\xb8\x07\xa6\xd3@\x1f:\x89VH\x8a\x08\xda\'IN_\x07s\x0e09\xca\xed\xe3\xa8\x19\xf9\x8eNG\x1e\x87\xf0\xfd\x9d\xf4\xadJ\xd1\xa5\x8d\xc2\xa26\xe4\x8c\xa8,F[\xa9\xe9\xc8\\c\x82\xa7\x96\x04\x1a\xe2\x1f\xecqO\xbaT\x9ee\xc0,\xa8\xc2=\xa7\x7f \x1c6F\xc1\xd7\x03\x04\xf4 |\xdd\xe7\xc3xU\xb4\xdb\xf9w\x06\x12E\xb9\x19H \xf1\x91\xf8P\x07\xa2\xe8W\x0c\x96\xf1A\x1e\x9d(\x89\xd8\xbbL\x860\x81\x9c\xbb\xbb\x11\xb81;\x87\'\x07&Py\xf9\x8a\xeeVF\x9f\r\xc4zlr\xdb6\xe9\x12\x07\tn\xec\x129\x1c\xe0\xa9f\xda\xcc\xb8\xc62;1\xc8c\x8cV\xff\x00\x847K\xff\x00\x9f\xads\xff\x00\x07\xb7\xbf\xfcz\x80>%\x12(\xb7x\x8c1\x97gV\x12\x92\xdb\x94\x00r\xa3\x9c`\xe4\x13\x90O\xca0G9\xf7\xef\x81Z\x8d\xcd\xed\x85\xc4W3I+F\xe8\x91\xb3\xb1;T.\xd0\xbe\xb8\x00(\x1e\x80b\xbc\x063\x08I\x84\xb1\xc8\xceS\x11\x15p\xa1[p\xe5\x86\x0e\xe1\xb7p\xc0\xc7$\x1c\xf1\x83\xef\x1f\x04/\xae/c\xbc\x92\xe2M\xd8\x90\xecEP\xa9\x18,X\xaa \xc2\xa2\xeef;T\x002x\xa0\x0fy\x81\xa62\xb2\xb2 \x80"\x94p\xe4\xb1l\x9d\xc0\xae0\x00\x1bprs\x93\xc0\xc745{ha\xd1\xaec\x86\x18\xe3C\xbeB\xa8\xa0\x02\xccK3q\xdc\xb1$\x9e\xe4\x93V\x85\xd7\x95qkl \x9eF\x9fq.\x89\xf2D\xaa9.\xc7\x81\x92T\x002\xc79\x03\x01\x8a\xb7W\x1b\xb4\xd9\xc7\xfb\x06\x80<#\xc4\xe2\xf2\x02L\xb7\x85\xbc\xe7ux\xe1\n\xb1\xed\xdc\x08_\x97\xa8\x05A\xeb\x82I#\x03h^n\'\x89n#\x13$\x8f\x1er\xca\x87i*\x08\xc8\r\x82\x01\xe7\x8c\x83\xf48\xae\xb3\xc5\x0e\xd6\xc2\t\xad\xa5\x91Z)s\x1c\x83\xe5e \xf0x<\x1f\xc6\xb95\x8aE\xb7\x8eGh\xce\xe2W\xe5a\x9c\x802v\xe4\x909\xe3>\xfc\x9c\x1a\x00tf0\xc7\xcdVe\xda@\n\xdbNppz\x1e\x01\xc1#\xb8\xe3#\xadu\x7f\x0f\'K}I\xad\x8a`I\'\xc9\xb1I\x19\xc1b[\x03\x0b\xc8<\x9fnrq\\\xb3:\xcd\xbd\xa5m\xae#U\x8cG\x12\x80\xc4`s\x8cc\x8c\x9c\xe0\x92z\xf5&\xbaO\x00\\\xc3k\xe2\x05\x13d\x87\x04\x00\x06y\xc7\x1d\xc7\x7f\xf2h\x03\xdf\xa1\xc8\x88\x020j0\xf1\xdf\xd9\xc35\xa5\xe7\xeed\xd9,s\xc0U\xc4\x89\x90\xdc\x12\x08*\xc3\x8c\x8e\xc7\x82\x0e\x08,gK\xab(g\x888I\x10:\x89\x10\xa3\x00FyV\x00\x83\xec@"\xac\xd0\x04f\xde\x16\xb8K\x86\x8a3:#"HTnUb\x0b\x00z\x80J\xa9#\xbe\xd1\xe9RQU\xed\xec\xe3\xb6\x9e\xeeT9k\xa9D\xaf\xf2(\xc1\x08\xa9\xd4\x00O\x089bOl\xe0\x00\x00\x0b4H\xe0e\x8e\xcf\xec\x8ae\x90\x98\xf0\xa3q.\xc4\xbf\xcaH\xf9\xc9/\xeb\xf3r\x01\xc8\xab\x15N\x19\xaed\xbd1J#\x8cF\x8c\xce\x8b\x1b\xb0`\\\x88\xd8JB\xaev\xa3\x16@\t\x05\x97\x9c`\xbd\x89\xa1Y\xd0#\x99\x00\x0e\xaf\xf2H\xc8r\xac\x18r\xa4\x1cdr:\x11\x90r\t\x14\x01\x1d\xcc\xb2[\xee\xb9f\xcd\xacQ;I\x1a@\xd2H\xc4`\x82\xbbNN\x00o\x94),H\xc61\x83,\xc6e\x8c\x18#\x8d\xdfz\x82\x1d\xca\x8d\xbb\x86\xe3\x90\x0f!r@\xee@\x19\x19\xc8\x8a\xec[$B\xea\xe2\x0f7\xec\x9b\xa6B\xb0\x99]\x08R\tE\x00\xb1m\xa5\x87\xca2rGz\xb3@\x15\xad\x9b\xed\x18\xbcIg\xf2f\x89\nC,~^\xce\xa7;J\x87\x0cC\x00Ct\xda8\x079\x96i\x1e(\xc3$2LK\xaa\xedB\xa0\x80X\x02\xdf1\x03\x00\x1c\x9e\xf8\x07\x00\x9c\x03\x1c\xd7~M\xe5\xb4\x06\xdevY\xf7\x012&\xe4F\x03 69\\\x8d\xd8$m\xf9pH%CGk>\x9bur\xd7\x16\x92\xdaM<\xb6\xf19\x92\x16Vg\x84\x971\x1c\x8eJ\x13\xe6\x15=>\xf6;\xd0\x04n\x92\xc3\xa9\xabM\xabl\x86yG\xd9\xed\x8a"\x96a\x1b\xee\x8c1\x19e\xc0\xf3\x00\x1f0(\xc7q_\x94[\x8c\xcc^a,q\xaa\x07\xc4E\\\xb1e\xda9a\x81\xb4\xee\xdc03\xc0\x07<\xe0KI@\x0bQ\xb4ln\x12Q3\x84Te1\x00\xbbX\x920\xc7\x8c\xe4`\x81\x82\x07\xccr\x0f\x18!\x9e\x1b\x94/\x04\xa9*\x07h\xcb#\x06\x01\x95\x8a\xb0\xe3\xb8`A\x1d\x88"\xa3\x89\x12+\xcb\x91\x1d\x9f\x95\xe6m\x96I\xc0P&r6\xf3\x83\xb8\xb0T@I\x1d\n\x80N\x08\x00\x19\xed\xa9\x1dJ\xd7L\x16W\x1eCj14\xd9F\x86I#\x88\xc4O\x98\xa7yS\xb5\xda!\xb9D\x8b\x96\x03\x046\xe1\xa9\x0c+\x04e\x10\xc8Av\x7f\x9eFs\x96b\xc7\x96$\xe3\'\x81\xd0\x0c\x01\x80\x00\xa2q3[\xca\xb6\xef\x1cs\x94"7\x91\x0b\xaa\xb68%A\x04\x8c\xf6\xc8\xcf\xa8\xa28\xd9\x1efi\xa4\x90;\xeeU`\xb8\x8cm\x03j\xe0\x03\x8c\x82y\xc9\xcb\x1eq\x80\x00\x01<-p\xf6\xeb,ftEw\x880\xdc\xaa\xc4\x85$u\x00\x95`\x0f}\xa7\xd2\xa4\xa8\xa73-\xbc\xado\x1aI8BcI\x1c\xa2\xb3c\x80X\x02@\xcf|\x1cz\x1a\x96\x80+\xec\xb8\xb8\xd3\xb6J\xff\x00d\xba\x92,3[\xb0\x93\xcar9(]p\xd8=\x0b/8\xe4v\xa9\x08\x9b\xed\x08\xcb$b\x00\x8c\x1d\n\x12\xc5\xb26\x90\xd9\xc0\x00n\xc8\xc1\xceG#\x1c\xc9E\x00SI\x16\xd2\xe0Z\xf97e$r\xc93\x16\x95K\xb1we\xceI@6\xff\x00\x16\x14nES\xfc!\xeb+\xc1\xe5\xc7r\xded\x92\xca\xea\x8d\x0c\x0c\x14\x0f\x99\x947-\xb7\n0X\x90\x0bt\xc1`\xb5,\x06f\xb7\x89\xae#\x8e9\xca\x03"F\xe5\xd5[\x1c\x80\xc4\x02F{\xe0g\xd0S\xe8\x029 \x86g\x89\xe5\x8a7x_|L\xca\tF\xdaW+\xe8v\xb3\x0c\x8e\xc4\x8e\xf5\x18\xb7\x92[8b\xbb\x97|\xcb\xb1\xa4x\x0bB\x19\xd4\x83\x90\x03\x12\x14\x91\xf7I9\x1c\x1c\x82sb\xa3\x92\x08fx^X\x91\xde\x17\xdf\x132\x82Q\xb6\x95\xca\xfa\x1d\xac\xc3#\xb1#\xbd\x00\x12L\xb1<(\xc2Be}\x8b\xb66`\x0e\xd2\xdf1\x03\n0\xa7\x93\x81\x9c\x0e\xa4\x03\x1c\xd7^_\xda\x16;y\xe7\x9a\x18\x84\xa24Ly\x99\xdd\x85Vl!bT\xf1\xbb\x8c\x82p\x085 \x13}\xa1\xd9\xa4\x8c\xc0QB B\x186N\xe2[8 \x8d\xb8\x18\x18\xc1\xe4\xe7\x82\x18V\x04(\x86B\x0b\xb3\xfc\xf23\x9c\xb3\x16<\xb1\'\x19<\x0e\x80`\x0c\x00\x05\x00\x12\xc6\xee\xf12\xcd$a\x1fs*\x85\xc4\x83i\x1b[ \x9cd\x83\xc6\x0eTs\x8c\x83\x8b\xe2\xd8\xa7\x9b\xc3\xf7\x0bnTI\xc7\xdeB\xc3\x19\x19\x18\xc8\xe4\x8c\x80{\x1epzV\xd4p\xa4O3\xa9\x90\x99_{n\x91\x98\x03\xb4/\xca\t\xc2\x8c(\xe0`g\'\xa9$\xd6\xd5J\x8d>m\xe4\x85\xdb\xc9\x03&\x80>l\x92\x0b\x985\x07\xb8WT{lK\x1b\x06\x08\xe8\xd9\xc6\xe59\xc9<\x8e\x9c\x8eOL\xe3\xd4>\x1f \x97E\x95\xf2\xa2C\x18\xdc\xa1\xb3\x83\x8eq\xd3#\xdf\x15\xe6\xba\xfa\xc9\xfd\xa9 \'`\x07*T\xe4\xb0\xe0\xf3\x91\xc7q\xdf\x8e\xfe\x9e\x9b\xe0(\x1a;\'\x85.-\x84\xb1\x0c\xdcF\x08\x90\x94;\x94m*\xd8\x19e\xc8nA\n\xc0z\x80\x0fB\xd3YZ\xc6<v\x18\xab\x95\x98\x89x\x9aAK\x0f \\\x9c\x04i\xf3\xb1ryb\x07-\x81\x93\xb7#v1\xb9s\xb8iP\x07\xc0\xb3\xc8\xb3\\K*C\x1c\x08\xeeYb\x8c\xb1T\x04\xfd\xd1\xb8\x93\x81\xd3\x92O\xa95\xec\xdf\x02\xafd{\x8b\x9bu\xb4U\x82%\x01\xe5M\xdf;\x92\xc5K\x12H\x07\x1c`c!:g$\xf8\xadz\xa7\xc1\x18\xd6]r\xe9X\xb8\x00#|\x8eT\xe40#\x90G\x19\x1c\x8e\x84d\x1c\x82E\x00}=d&\xc4\x86GFB\xc0\xc4\x15\n\x95]\xa3\x869;\x8e\xed\xc7#\x1c\x101\xc6J\xdf\xc2g\xb5d\x12\xbc\\\xab\x16@\xa4\x90\x08%y\x04`\x81\x83\xdf\x04\xe0\x83\x82\x1djs\x10\xfaU{\x98a\x92W\xb6\xb9\xdfr%o\xb4F\x92\xc2\x1a8\xbc\xbd\x98\x01\x82\xe0\x10\xd8q\xb8\x96\xceH\xe1~P\x0f\x1c\xf1P\x9b\xec\x97\x91G\xb0[\xabopd\xdaw\x02\x02\xe1\x7f\x8b\xef7\xb8\xf4\xc6H\xe1v\xa9`\xe5F\xe0\x08\x07\x1c\x80z\xff\x00!\xf9W_\xe2\x0b\x91sk|\xc2\xee)\x95\xa52\xc5$1\xe5$\x88\xe3`\xce\xee\xbdN\xf1\x90x\x01GZ\xe4\xd1\xd9\x03\x14 3#&J+c \x8c\x80\xc0\x8c\xf3\xc1\xc7\x07\x06\x80\x1dn\n\xdc\x17Yb\x8f1\xb8o9\x0b\xa9\x1b[\x800@bN\x03c\x83\x8eF25\xbc\x1cL\x9e(\xb5\xc1\xc2\xee*AR\t9\xeb\xf4\xeb\xdb\x9e?\x1ci\xca,\xd2%\xb9i\x17\xe61\x99\x17f\xe0:n\xc6B\x93\x91\xc6O~\xb8\xad]\x12\xce\'\xd6c\xb4\xd4\xa1\x7f\xb3\xdc&\xc9\x11\xb2\xa5\xa3\x91\x7f\x03\x82\xad\xc1\x1d\x8eE\x00}\x13-\x94w\xbak[9\xdb\x90\nH\x11X\xc6\xe0\x82\xae\xa1\x81]\xca\xc00\xc88 qN\xb4\x83Mb.,\xe2\xb4%\x1ed\x12B\xab\xf2\xb3I\x99\x86GBd\\\xb0\xee\xcb\xcf"\xa7\xb7`b\x18\xc1\xfaR\xb4*\xd7\t92oDd\x00H\xc1p\xc4\x13\x95\xce\t\xf9F\t\x19\x1c\xe3\x199\x00\xcf\xc6\xa4\xb7\xb77\x97\x16\xd1\xbcP\xdb\xb2\xdb[\xda\xdc1y\x1b{\x96\xdc\x18\xacy*\xb0\xed\xcf\xdd&A\xbb\x07\'B\t\x96\xe6\xde)\xd0H\x12D\x0e\xa2H\xd9\x18\x023\xca\xb0\x05O\xb1\x00\x8e\xf5Ml\xe3\x8fQ\x8c\xdbYA\x02\xc5\xbd\xda\x7f!N\xf1)f\x91\x10\x86\x05X\xba\xa3\xb1 \x86\xe3\xa9\xc9]\n\x00\x8eHVW\x89\xd8\xc8\x0cO\xbdv\xc8\xca\t\xdaW\xe6\x00\xe1\x86\x18\xf0r3\x83\xd4\x02\x08\xe0\x86\x17\x95\xe2\x8a4y\x9f|\xac\xaa\x01v\xda\x17-\xeav\xaa\x8c\x9e\xc0\x0e\xd5]\xac\xe6me/Z\xeeA\x04v\xed\x12[.B\x96f\x05\x9d\xf9\xc3\x10\x11B\xf06\xe5\xf9;\xbe[\x94\x01\x1c\xe6e\xb7\x94\xdb\xc7\x1c\x93\x84&4\x91\xca+68\x05\x80$\x0c\xf7\xc1\xc7\xa1\xa9(\xa2\x80 \xdb\xf6\x89\xf3$S\xc5\xf6is\x1by\x9bV\\\xa63\x85o\x99~r0\xe3\xef.q\xc2\xb5I$L\xf2B\xcb4\x91\x88\xdfs*\x85\xc4\x83i\x1b[ \x9cd\x83\xc6\x0eTs\x8c\x82\xfa(\x01j\x9a\xdd4\xf7\xefm\x13\xa4On\xea\xd3G&\xd6i"d;YB\xbeP\x17\x04e\x87>[\x80:5[\xa2\x80\x16\x8a\xafn\xdb\xa7\xbb\x1el\xef\xb6P6\xcb\x1e\xd5O\x91N\x10\xed\x1b\x97\x9c\xe7-\xf3\x16\x19\xe3h\x8e\xd6y\x9d\xd6\x06\x96\xd2\xe1\xe1M\x97\x92D\xc5JM\xb5\x18\x01\x1f\xcd\xb4\x15b\xd8-\x90\n\xfd\xed\xd9\x00\x16&\x82\x1b\x98\xc2O\x14r\xa0up\xae\xa1\x80e`\xcay\xee\x18\x02\x0fb\x01\xa9*\xb5\x8d\xe4Z\x85\x9cwP\xac\xe9\x1b\xe7\x0b<\x0f\x0b\xf0H\xe5\x1c\x06\x1d;\x8ez\xf4\xa6)\xbb\x99\xec\xaeV9!FB.-\xa6t\x057.A;Cnu`\x17\x01\xc2\xe1\xd8\xe5\xb0\xb4\x01<\xf7\x10\xda\xdb\xcbqq*C\x04H^I$`\xaa\x8a\x06I$\xf0\x00\x1c\xe6\xa3\xdd\xbfQ\xda%\x9d|\xa8\xb2\xd1\xf9x\x8d\xf7\x9e\x0e\xe2\xbc\xb0\xd8\xdc\x06\xe06Xr\xa6\xa4\x9ae\x81\x03\xb8\x90\x82\xea\x9f$l\xe7,\xc1G\n\t\xc6O\'\xa0\x19\'\x00\x13D\xf1\xb4\xd6\xf2\xc4\x93I\x03\xba\x15Yc\nY\t\x1fxn\x04du\xe4\x11\xea\r\x00\x11\xca\xee\xf3+B\xf1\x84}\xaa\xccW\x12\r\xa0\xee\\\x12q\x92G89S\xc60I\x1c+\x13\xca\xead&W\xde\xdb\xa4f\x00\xed\x0b\xf2\x82p\xa3\n8\x18\x19\xc9\xeaI1\xd9\xdf\xd9\xea\x11\x19l\xae\xe0\xb9\x8cm\xcb\xc3 p7(u\xe4z\xab+\x0fP\xc0\xf45b\x80+\xdb=\xc8\xdb\r\xcao\x91"B\xf7\x08\xa1#\x91\xceC\x05]\xcc\xcb\x8cg\x07\xb3\x0c\x169\xc4\x91\xcf\x0c\xcf2E,n\xf0\xbe\xc9UX\x12\x8d\xb46\x1b\xd0\xede8=\x88=\xe8\x98\xcc\xa8\x0c\x11\xc6\xef\xbdA\x0e\xe5F\xdd\xc3q\xc8\x07\x90\xb9 w \x0c\x8c\xe4I@\x05\x14Q@\x11\xc3#J\x85\x9e\x19!!\xd9v\xb9RH\x0c@o\x94\x91\x82\x06G|\x11\x90\x0e@\x8e\xe2\xc6\xde\xeaX\xe4\xb8\x8f\xcd\xf2\xf9Tv%2\x19\\1O\xbaX2)\x0cFW\x1c\x11\x93\x9b\x19\xa8\xa7\x9dm\xad\xe5\x9d\xc3\x94\x8d\x0b\xb0\x8e6v \x0c\xf0\xaa\tc\xec\x01\'\xb5\x00+B\x8dp\x93\x93&\xf4F@\x04\x8c\x17\x0cA9\\\xe0\x9f\x94`\x91\x91\xce1\x93\x9c\xfdg\xcd6\x92 \x8d\x0c\x06\'.\xe5\xce\xe0\xdcm\x01q\x82\x08\xdd\x93\x91\x8c\x0e\x0ex\xb9\x12\xf9\xf2\xad\xcc\x91O\x0c\x91\xf9\x91*<\x9c\x15\xdc>b\xaa\xc5Nv\x02\t\xf9\x80b>RXS.\xa6\x80\x93l\xd2\xc6&\x927u\x88\xb0\xdc\xca\xb8\x0c@\xea@,\xb9=\xb7\x0fZ\x00\xf9\xcb\xc4\xc7n\xaaA\x18\xca\xd7\xacx\n8o\xbc+%\xa4\xd0\x89-\xca\x18\x9a9v\xb2J\xac\xa3w\xcb\x93\x90rT\x82\x07C\xc60O\x97x\xb6C%\xf2\x01$rI\x011\xdcb"\x9f8\x1c\xe1I\xca\xe7\x822NT\x82\t\x04\x1a\xf4\xdf\x86\t\x8d\x022x\xceh\x03\xbb[x.-\x929\xa2IQY\\+\xa8`\x19X2\x9e{\x86\x00\x83\xd8\x80j\xcdT\x13=\xad\xbd\xd4\xb3\xc5\x88a\xcb\'\x92\x1aWt\n\t;\x02\xe7vw\x00\xab\xbb8\x1d\xce\x05\xba\x00\xf8\x14\x08~\xce\xec\xd2H\'\x0e\xa1\x10 *W\x07q-\x9c\x82\x0e\xdc\x0c\x1c\xe4\xf21\xcf\xa3|\x18\x9eX|P\xfb<\xaf-\xc0G\xcb\xa8|\x9c\x91\x85\xceH\xe0\xe4\x81\x81\x91\x9cdg\xcdk\xbc\xf8Dq\xe3\x88\x7f\xdc4\x01\xf5\\M\x03X\xaf\xf6\xa2\xda*\x0b\x85\t\xbd\xc3!o4y$\x16\x03\xe7-\xb0\x81\x8e\x18\x80\t\xc0\'BE\x98\xce\x8c\x1d\x04\x01\x18:\x14%\x8bdm!\xb3\x80\x00\xdd\x91\x83\x9c\x8eF9\x86\xc4\xe6%\xe3\xb5N\xe2o=\x18<~@F\xde\x85\tb\xd9\x1bHl\xe0\x007d`\xe7#\x91\x8e@<s\xc6V\x91\x8f\x10]\xe3lQ\xbcd\xfc\xd9 \xb0\x19\xec;\x9e\x07\xd7\x9fZ\xf3\xf1r"\xb2\x92\x03\x0cm\xe6H\x8d\xe6l%\xd7\x1b\x86\x01\x1d\x14\xee\xc9\xff\x00t\x1e\xd5\xe9\xbe9\xb6\x8eMX\xc8\x19\xfc\xc0\xac\xa4d\x85\xc1\xc7=y<w\x1cv<\x9a\xf3\x15\x9f\xec\xf7\x05Ur\xc4\x15\xf9\xa2\xdc\xb8`Gp@8\xcf=A\xc7C\x8a\x00X\x91d\x95\x11\xa4X\x95\x98\x03#\xe7\n=N\x018\x1e\xc0\x9a\xbd\xa1\xb0\xfe\xdc\xb5^w\x16\xc8\xe0\xe3\xa8\xef\xf8\xd5\x08G\xda$\x81b*\xc2f\x01\\\xba\xaa\x0c\xf4%\x89\x00\x0f\x7f\x7fNk[Kh\x8e\xa5k,\t\xfb\x9bvH\xe6\x97i\x1f\xbc`[\r\xc9\x00\xf5\x03\xa6UA\xc09\xa0\x0f\xa2\xecq\xf6X\xff\x00\xdd\x15(\x13}\xa1\xd9\xa4\x8c\xc0QB B\x186N\xe2[8 \x8d\xb8\x18\x18\xc1\xe4\xe7\x8a\xbaLpE\xa7\xc6-\xa3\x8d#l\xb9\x11\xa8\x00\xb3\x12\xccx\xeeI$\x9e\xe4\x93W\xa8\x028 \x86\xd6\xde+{x\xa3\x86\x08\x90$q\xc6\xa1U\x14\x0c\x00\x00\xe0\x00;T\x94U{\x1b_\xb1Y\xc7o\xf6\x89\xee\x19rZi\xdfs\xbb\x12I\'\xb0\xc9\'\x80\x02\x8e\x80\x00\x00\x00\x12M\x1bJ\x81Ri!!\xd5\xb7 RH\x0c\t_\x98\x11\x82\x06\x0f|\x13\x82\x0e\x08\x92\xab\xb4\xafo\xe6Ip\xdb\xe3iQbX`b\xca\x1bj\xe1\xb0N~bIl\x00\x14\xf3\xc2\x962C\x046\xc8R\x08\x92$.\xceU\x14(,\xccY\x8f\x1d\xcb\x12I\xeeI4\x01%\x14f\xab\xad\xfd\x9b\xf9{n\xe0o6W\x82<H\x0e\xf9\x13v\xe4\x1e\xac6>GQ\xb5\xbd\r\x00HaV\xb8I\xc9\x93z# \x02F\x0b\x86 \x9c\xaepO\xca0H\xc8\xe7\x18\xc9\xc9 \x98\xbcF)#T\x0f\x99C!b\xcb\xb4\xf0\xa7#i\xdd\xb4\xe4\xe7\x80F9\xc8#\x85by\x9dL\x84\xca\xfb\xdbt\x8c\xc0\x1d\xa1~PN\x14aG\x03\x039=I&J\x00(\xa2\xa3\x8e\x08a\x92Y"\x8a4y\x9f|\xac\xaa\x01v\xda\x17-\xeav\xaa\x8c\x9e\xc0\x0e\xd4\x01%W\xf3\x12\xe6}\xb0^a\xade\xc4\xf1\xc6T\xe4\x94\xc8G\xc8$p\xea\xfc`\xfd\xdepH+f\xe9<F\xea\x1b\xcf\xb5[\xdc\xed\x96\x16R\xac\x81\n\xae6\x15\x1c\xa9\xc6\xec\x92~\xf1\xe7\x18\x02\xc5\x00S\xf2&\xb8\xb8\xb8\x8e\xf6+I\xac\xc3\xc5-\xbf\xcaK+)\x07\xe6S\x91\x95e\x0c\xac\x08\xeb\x8c\x02\x9b\x99$\x86\x19.\r\xbd\xd7\x99r%qq\x1aK\x00h\xe2\xf2\xca`\x06\x0b\x80Ca\xc6\xe2[9#\x85\xf9l\xc9\nK$N\xc6@b}\xeb\xb6FPN\xd2\xbf0\x07\x0c0\xc7\x83\x91\x9c\x1e\xa0\x11%\x00%\x15\x1c\xe2f\xb7\x95m\xe4\x8e9\xca\x11\x1b\xc8\x85\xd5[\x1c\x12\xa0\x82F{dg\xd4TwVq^y\x1ek\xce\xbeL\xab2\xf93\xbcYa\xd06\xc27/<\xabeOph\x02A\x1b\x0b\x87\x94\xcd!FEQ\x11\x0b\xb5H\',8\xceN@9$|\xa3\x00s\x99*1\x04+p\xf7\x0b\x14bwEG\x90(\xdc\xca\xa4\x95\x04\xf5 \x16b\x07m\xc7\xd6\xa3\x87\xca\xba\xfb=\xf2y\xeb\x98\x8e\xc5}\xf1\xfc\xaf\xb4\xfc\xd1\x9ca\xb8\x1fxn^G\x19 \x80X\xaa\xf7\x12F\x93\xda\x07\xbc\xf2\x19\xe5!#\xca\x8f\xb4\x1d\x8cvr2p\x01\x7f\x97\x07\xe4\xeb\x8c\x82\xf8\xdac$\xc2H\xe3T\x0f\x88\x8a\xb9b\xcb\xb4r\xc3\x03i\xdd\xb8`g\x80\x0ey\xc0\x04\xf0\xb5\xc3\xdb\xac\xb1\x99\xd1\x15\xde0\xc3r\xab\x12\x14\x91\xd4\x02U\x80=\xf6\x9fJ\x00$\x9dbx\x91\x84\x84\xca\xfb\x17ll\xc0\x1d\xa5\xbeb\x06\x14aO\'\x038\x1dH\x064\xb5\xf2\xa2\xb5\x8a+\x89\xd6;|\x0c3\xf9\x86U\nT\x07g\xcb\x1e\xa1\xb3\x9d\xc4\xa8\xc9# \xc9\x04\xcbso\x14\xe8$\t"\x07Q$l\x8c\x01\x19\xe5X\x02\xa7\xd8\x80Gz\x8e\xf2\xd7\xed\x90,_h\x9e\r\xb2\xc7.\xf8\x1fk\x1d\x8e\xaf\xb4\x9f\xee\xb6\xdd\xac;\xa9#\xbd\x00I \x98\xbc&)#T\x0f\x99C!b\xcb\xb4\xf0\xa7#i\xdd\xb4\xe4\xe7\x80F9\xc8\'\x82\x1b\xabym\xee"\x8eh%B\x92G"\x86WR0A\x07\x82\x08\xe3\x15^\xda\r4\\9\xb5\x8a\xd3\xcf\xb7y\x11\xccJ\xbb\xa2i\n\xca\xe0\xe3\x90X\x94r;\xe4\x13\xd8\xd5\x88#hm\xe2\x89\xe6\x92wD\n\xd2\xc8\x143\x90>\xf1\xda\x00\xc9\xeb\xc0\x03\xd0\n\x00\x92\xab<\x8c\xebp\xad\x0c\x91\x84;U\x98\xae$\x1bA\xdc\xb8$\xe3$\x8epr\xa7\x8c`\x9b4\xc7\x00\x82\r\x00x\xbf\x8f\xf4\xcf\xdc5\xd8\x1d\x1b\x15\xdc\xf8\x01\x15|;fB\x84c\x1a\xee]\xfb\xbels\x83\x81\xc6s\xda\xb3\xbcn\x854{\xb8\xc4\x0eP\xa37\x981\xb5H#\x00\xf3\x9c\x9c\x9201\xf2\x9c\x91\xc6m\xfc:F_\rY\xb3!\\\x83\x83\xcf\xcd\xc9\xe7\x9f\xcb\x8fJ\x00\xee\x05`}\xb3\xc5\xff\x00\xf4\x02\xd0\xff\x00\xf0s7\xff\x00"\xd6\xf8\xa5\xa0\x0f\x80k\xb3\xf8_y\x15\x9f\x8c\xe02o\xdc\xe3laT\x10X\x91\xd4\xe4`c\'<\xf2\x00\xc79\x1c\xa3\xca\x87N\x86 \xd9\x91e\x91\x99|\x85\x18\x04 \x07\xcc\x07st?)\x18\\dr\xcd\x8d\xbf\x01\xac\xcd\xe3-<\xc3o,\xec\xaf\xbc\xc7\x10\x05\x8a\xa8\xcbc$\x0c\x85\x04\xf2@\xe3\x92\x07 \x03\xec\xad9\xb3\n\xfd*ux\xda\xfex\xc5\xe6\xe9\x041\x96\xb5\xca\xfe\xec\x12\xf8|\x01\xbb\xe6\xc1\x1c\x9c~\xef\x80\x0e\xec\xe7\xe8V\xdfg\x82Bg\x9af\x96C!2\xbeq\x9e\x02\xa8\xe8\xaa\x00\x00\x00;d\xe5\x89\'g\xb5\x00y\xc7\x8d,-\xe0\xbe\xf3\xa3\x89\x15\xe7;\xa5*0\\\xe0.O\xa9\xc0\x03\xe8\x07\xa5y=\xfc)\x05\xf4\xb0\x01"\xca\x8c\xcb*H\x9bv0b1\xd7\x9e1\xd7\x1c\x921\xc6O\xafx\x9do~\xc7g&\xa1\x14\x12^-\xba\x9b\xaf 7\x94\xb2\x10\x03\x15\xcf;C\x1e3\xed\x9fJ\xf1\xdb\xa2\x06\xabp\xbc\xe4\xb1=\x0e8\xf7\xfch\x01\xafs\xe7~\xefl\x1e`\xd8X\x85\xc3\x04\nT\x00\x01\x03\x9c\x02I\x04\x92\xbdrI.\xb6H\x84\xe5\xd9\xe4\x12\x10\xa1\x00\x19\x04\xee\xef\xcf\x1c\x12r\x01\xe8\x07\xb8D\x92H\xd6EGeY\x17k\x858\x0c2\x0e\x0f\xa8\xc8\x07\xea\x05$f1q\x17\x992B\xad"\xa9\x92L\xed\\\x902p\t\xef\xd8\x13@\x1fF\xf8^5\x8bB\xb7U.A\x05\xbew,rI\'\x92O\x19<\x0e\x80`\x0c\x00\x05j\xc9<0\xbcI,\xb1\xa3\xcc\xfb"V`\x0b\xb6\xd2\xd8_S\xb5X\xe0v\x04\xf6\xac\x9f\x0c8m\n\xd0\xa9\xcf\xee\xc5l\xd0\x03 \x85-\xad\xe2\x823!H\xd0"\x99$gb\x00\xc7,\xc4\x96>\xe4\x92{\xd20\x9b\xed\x08\xcb$b\x00\x8c\x1d\n\x12\xc5\xb26\x90\xd9\xc0\x00n\xc8\xc1\xceG#\x1c\x84\xcd\xf6\x84U\x8e3\x01F.\xe5\xc8`\xd9\x1b@\\`\x827d\xe4c\x03\x83\x9e$\xa0\n\xe9h\x90Ek\r\xa9\xfb-\xbd\xb6\x02\xc1\n*\xa1@\xa5Bc\x1c(\xc8#n>\xe8\xe7\x19\x06\xc5\x14P\x01EG\x0c\x8d*\x16xd\x84\x87e\xda\xe5I 1\x01\xbeRF\x08\x19\x1d\xf0F@9\x028m\xde+\xcb\x99|\xdd\xd0\xcd\xb5\x82\x12\xc4\xab\x81\x86 \x96 )\x010\xaa\x06\x08br[\x80\tm\xdaf\xb7\x89\xae#\x8e9\xca\x03"F\xe5\xd5[\x1c\x80\xc4\x02F{\xe0g\xd0Q\x1c\xf0\xcc\xf3$R\xc6\xef\x0b\xec\x95U\x81(\xdbCa\xbd\x0e\xd6S\x83\xd8\x83\xde\x90\xc1\x0b\\%\xc3E\x19\x9d\x11\x91$*7*\xb1\x05\x80=@%T\x91\xdfh\xf4\xa8$\xd4!\x83T\x86\xca\xe2{H\x9e\xe53j\x8d8\x12\xce\xcb\x93 T#\x90\xab\xb4\xe4\x13\xd4\xe4\x0c\x02@.QQ\xcd\x1bJ\x81Ri!!\xd5\xb7 RH\x0c\t_\x98\x11\x82\x06\x0f|\x13\x82\x0e\x08\x92\x80\nZ\xcb\xd2^\x1b\x8b\x9dB\xe15h\xf5\t\x05\xc3A"\xc3 1\xdbl\'\x11l\x04\x85p\x1b,O\xccI\xe7\n\x15V\xe4\xabxec\x14\xf0,\x7f\xbb\xda\xaf\tb0\xc4\xc9\x93\xb8g+\x808\xf9O\'p\xf9@\x01rm\xed\xb3\x7fs?\x93\x1d\xbcN]\xdeb\x91\xaapY\x98go\x1b~\xf1\xe83\x82\x019\x92e\x99\xa3\x02\t#G\xde\xa4\x97B\xc3n\xe1\xb8`\x11\xc9\\\x80s\xc1 \xe0\xe3\x068\xaee\x92UF\xb2\x9e5>f]\xcaav\xb0U\xce\x18\x9f\x9c\x1d\xc3\x8e\x83\xe6\xdax\xab\x19\xa0\x04\xaa\xe9\xe5Ax\xd1\x8f<\xc9q\x99\x89;\xdd\x06\xd0\x8a@\'*\x9f\xc3\x85\x18\xcf\xcc@\'q\xa2i\xbe\xc9\xf6\x8b\xab\xab\x98"\xb1\x8e \xe4\xb8\xdb\xe5\xed\xdc]\x99\xc9\xc6\xdcm\xec1\x82I9\xe2\xc5\x00W\xbc\xb1\xb7\xbe\x88Gq\x1e\xed\xbb\x8a:\xb1W\x8c\x95(Y\x1ca\x91\xb6\xb3\r\xcaA\x19<\xd5\x83T\xe7\x9ed\xd5,\xe0\x8a[M\x92$\x8d,21\x12\x95]\xbf:z\x85b\xaa\xc0\x8f\xf9h\x0e\xe1\x8d\xafn\x80\n\xafd\xdee\x9cr\x89g\x91e\xcc\xaag\x8f\xcbp\xacK\x05+\xb5J\xed\x04.\x08\xdd\xc7995b\xab\xb7\xdb\x13\xcc+\xe4K\x99S\xcbS\x98\xf6G\xf2\x86\xc9\xf9\xb70\xf9\xd8p\x01\xf9W\x8eZ\x80,TsA\r\xca\x04\x9e(\xe5@\xea\xe1]C\x00\xca\xc1\x94\xf3\xdc0\x04\x1e\xc4\x03D\xf3\xc3ko-\xc5\xc4\xb1\xc3\x04H^I$`\xaa\x8a\x06I$\xf0\x00\x1c\xe6\xa9\xbc\xda\xa3\xb5\xd7\x93k\x04~D\xa4B&\x93"\xed<\xa0A\xdc\xbc\xc5\xfb\xc6\xdaIV\xe1\t\xc7\xcc0\x01\xa1Q\xc32\xce\x85\xd0H\x00vO\x9e6C\x95b\xa7\x86\x00\xe3#\x83\xd0\x8c\x11\x90A\xaa\xf1\xc7<\xe934\xd7v\xe5\xae7*\xb0\x88\x95D`6\xae\x01\x1b\x1c)<\xe5\x80\x90\xf2\xa4\x00\xb7(\x00\xaaV\xf6\xe2\xde\xf6x\xe1\x89\xe2\xb7(\xae\x15V5\x88\xc8\xcf#9\x18\xf9\xf7\x92Ab\xdc\x1c\xa9\x1c\xef\xab\xb4P\x07+\xe2\xfb(\xe2\xf0\xd5\xccj]\x8339.\xec\xe7,\xc5\x8f$\x93\x8c\x9e\x07@0\x06\x00\x02\xa6\xf0\xbd\x9c\x90\xf8z\xd5m\xda8\xdf(It,6\xe4n\x18\x04rW \x1e\xc4\x83\x83\x8c\x1d\x1dz5\x97H\xb8VP~By\xaa\xbe\x14F]\x12\x1d\xe4\x93\xef@\x1bSB\x93\xa0G2\x00\x1d_\xe4\x91\x90\xe5X0\xe5H8\xc8\xe4t# \xe4\x12*J\x8e8V\'\x99\xd4\xc8L\xaf\xbd\xb7H\xcc\x01\xda\x17\xe5\x04\xe1F\x14p03\x93\xd4\x92d\xa0\x0f\x80k\xa4\xf0\x1d\xf4Zw\x8cl\xae&\xf2v\x01"\xfe\xf9\x0b.J0\x18\x00\x1f\x9b\'\xe5=\x03`\x9c\x00Ma\xcc\xef-\x9d\xb1\x92\xf3\xcd\xf2\xf7E\x1c\x04\xb10\xa0;\xb8\xc8\xda\x14\xb3\xb9\x00\x1e\xa1\x89\x03 \x99\xb4_\xf9\r\xd9\x7f\xd7e\xfet\x01\xf6\x8d\xad\x9d\xa6\xa9\xa5\xfd\x97P\xb4\x86\xea\xdaUR\xf0\\D\x1d\x18p\xc3*F\x0f8?\x95k7\x95e\x15\xd5\xd1\xf3\xd9Nf\x90.\xf9O\n\x06\x11\x06OE\x1f*\x8eNN\t\'8z1\x95V\xc9\xd7\xcfh\xca\x98\xdd\x13f\xc5\xc8\xc8v\xcf\xcd\xc6\xdd\xa3i\xfe>A\x1c\xaf@\x86o\xb4:\xb2F \x08\xa5\x1c9,[\'p+\x8c\x00\x06\xdc\x1c\x9c\xe4\xf01\xc8\x07!\xe3m:\x16Qx\xa8\xa2vP\x8c\xe1F\xe2\xaaI\x00\x9e\xe0\x16l}O\xadx\xb5\xf2\x98u\x0b\x80\xc1\x18\xb2\xb8\x04\x021\x9c\x85#\x9e\xab\x90}\t\x1c\x8cq^\xf9\xe2\xc0\xa7N\xf9\x81#\x9e\x01\xc78\xe3\xf5\xaf\x08\xd6\xa3T\xbeYK\xaeX2\xaa\t\x06\xe2\x06\xdc\x92\xb9\xce9^H\xf5\xc7z\x00\xcfe\x0e0I\xe0\x83\xc1#\xa1\xcfj[e\xb5\x8a(-\xccsC\x1cm\xb1\xa4R\x1f\xf7x\\m\\\x8eG\xcd\xd5\x86x\xe9\xd4\xc6\x9b\xdc\xab\x12Qp\x0e\xcc\x0c\xe7\x07 \x9e}GN\xe3\xa9\xa9\\\r\xb4\x01\xed~\x03x\x0f\x84m\xae\xeea2\xfd\x93t\xc8DFWB\x15\x81(\xa0\x16-\xb4\xb0\xc2\x8c\x9c\x91\xde\xbb\xaa\xe2>\x17\xe1|%\x02\x02p\x84\x81\x93\x93\xf8\x93\xc9\xae\xcey\xe1\xb5\xb7\x96\xe2\xe2X\xe1\x82$/$\x920UE\x03$\x92x\x00\x0e\xf4\x00C#J\x85\x9e\x19!!\xd9v\xb9RH\x0c@o\x94\x91\x82\x06G|\x11\x90\x0e@\xa7s\xa6\xc5$Q[\xc3\x0f\x95\x03K#\xcb\xe4\\=\xb1\x1b\xd5\xf70\xf2\xf1\xb9\x8b>y#\x93\xbf;\x94U\xc8aH\x10\xa2\x19\x08.\xcf\xf3\xc8\xcer\xccX\xf2\xc4\x9cd\xf0:\x01\x800\x00\x15%\x00\x14Tq\xc8\xce\xf3+B\xf1\x84}\xaa\xccW\x12\r\xa0\xee\\\x12q\x92G89S\xc60L\x94\x01]-\xe5[\xc6\x9c\xde\xce\xd1\x9c\xe2\x02\x13b\xe4 \x18!wq\xb5\x88\xc9\xff\x00\x96\x8d\x9c\x8d\xa1lTpN\x976\xf1N\x81\xc2H\x81\xd4I\x1b#\x00FyV\x00\xa9\xf6 \x11\xde\x88\xdab\xf3y\xa9\x1a\xa0|DU\xcb\x16]\xa3\x96\x18\x1bN\xed\xc3\x03<\x00s\xce\x00\x01\x04\x10\xda\xdb\xc5oo\x14p\xc1\x12\x04\x8e8\xd4*\xa2\x81\x80\x00\x1c\x00\x07\x18\xa9k:\xda;k\t\xd5dX"\x92M\x96\xb6\xf2\xbc\xe6I\xae\x11\x10\xb0Vf\x1b\x99\x87\xefN2\xdc\x02\xd9\xc9 [\xfb\xb7/!\xb8r\x8c\x8a\xa2#\xb7j\x90NXq\x9c\x9c\x80rH\xf9F\x00\xe7 \x13Ts\xc2\xb76\xf2\xc0\xe6@\x92!F1\xc8\xc8\xc0\x11\x8e\x19H*}\xc1\x04v\xaa\xb7w:z\xcb\x03\\\xb5\xb9\x92\'/\x11\x93\x04\xa3m+\xb9s\xd0\xedf\x19\x1d\x89\x1d\xea\xb5\xff\x00\x88,\xedm]\xe2\x9e\xdd\xe6\xda|\xb4y6\xab68\x05\x80$\x0c\xf7\xc1\xc7\xa1\xa0\r9\xa1Y\xd0#\x99\x00\x0e\xaf\xf2H\xc8r\xac\x18r\xa4\x1cdr:\x11\x90r\t\x14\x19\x95n\x12\x02$\xde\xe8\xce\x08\x8d\x8a\xe1H\x07-\x8c\x03\xf3\x0c\x02ry\xc6pq\x83/\x8c\xb4\x94\x91bI\x9aI\x19K(Db8\xc7|`\x1eG\x07\x93\xce:\x1aG\xf1}\x94Q\xab\xca\xaf\x1a\x96U\x05\xc62I\x00\x0f\xa9$\x01\xeeh\x03~\x112\xa1\x13\xc9\x1b\xbe\xf6 \xa2\x15\x1bw\x1d\xa3\x04\x9eB\xe0\x13\xdc\x82p3\x80C\x046\xd1\x94\x82(\xe2B\xec\xe5QB\x82\xcc\xc5\x98\xf1\xdc\xb1$\x9e\xe4\x93\\\xe4\x1a\xf5\xe4\x97\xd3\x17H\x16\xc8D\x9eW\xcd\x897\xe5\xb7\xe7<c\x1e^1\xcf\xde\xcfj\x82\xef\xc5\x10Gp\xb0}\xb6\x18X\xfc\xcd#\x00\xca\xa02\xe5\x08\xdc\x0e\xe6R\xd8 \x10\x08\xc9\xec\x08\x07[\xb8\x0e\xf4\xc9\x1aa$"$\x8d\x90\xbe%,\xe5J\xae\xd3\xca\x8c\x1d\xc7v\xd1\x83\x8e\t9\xe3\x07\x87\xd5<{\xa2\xd9Ol?\xb4%\x93\x0f\x92!P\xca\xff\x00+|\xad\x90p2s\x91\x8eT\x0c\xe0\xe0\xd1\x1f\x15l\xa6\x8a@\x90\x14\x95\x1c\xa8\xcb\x1d\xb8\xcf\xcar@\xceW\x19\x03\xa1$d\xe3\x90\x0fE\x9d\x1akybI\x9e\x07t*\xb2\xc6\x14\xb2\x12>\xf0\xdc\x08\xc8\xeb\xc8#\xd4\x1aFV7\t/\x9d"\xa2\xa3)\x88\x05\xda\xc4\x90C\x1e3\x91\x82\x06\x08\x1f1\xc8<c\xcb\x9b\xe2c\xad\xa2\x89\xa7\xb72\x85\x01\xde%#\'\xb9\x00\x93\x81\xed\x93\xf55\'\xfc,\x0bh|\xbf\xb4]\\\x89\x1a5\x90!\x88\x8cg\x91\x9c\x81\xc1\\0# \x82?\x00\x0fFMAe\x8e\xdaX\xa1\x99\xa3\x9f\x04\x96_,\xc4\xa5K\x02\xea\xf8a\xc8\x0b\x8cn\x05\x86@\xe4\x85k\xf8\x83\x15\x0c\t\x1e\x95\xe5\x96\xfe<\xb7DAp#\x92@\xa3{\x8c\xa8f\xeeB\x92p=\xb2~\xa6\xac\x1f\x88\xd6i0@UT\xff\x00us\xfa\xd0\x07\xa7-\xce\xf1\x95REL\t#\x9a\xf3m3\xc7\x10\x88\x9d\x01a\x89\x9c\x9d\xce\xf2gs\x16\x04\x16\x1d0\xc3\x81\xc2\xfd\xd1\xc2\xd6\xda\xf8\x9d\xaec\x06\xd8+\x12\xea6\xbb\x15\xc2\xe4d\xe7\x07\x902@\xc7$c\x8c\xe4\x00u\xf9\x15\x1d\xba\xb4V\xf1D\xf3\xbc\xee\x88\x15\xa5\x90(g }\xe3\xb4\x01\x93\xd7\x80\x07\xa0\x15\x92\xb7\xcd\xb43\xb2\xaa\x8f\xef5E\x0e\xa7\xa6\xa5\xd3M\xe7\xdb\t\xa4U\x8d\xa4\x18\x0c\xca\t*\x0bz\x02\xccFx\x1b\x8f\xa9\xa0\x0e\x82\x93"\xb9\xc9<a\xa6\xc6\xec\x81\xd9\xc8\xee\xa2\xa4\x83]\xfbf\xe3o\x04\x8d\x8eA*p\xdf\xfd\x7f\xf3\xf5\x00\xb9\xa9;O\xa4\xceZ\x19!l\xba\x85r\xa4\x90\x18\x80\xdf)#\x04\x00G|\x11\x90\x0e@4\xef6\x1d\x18}\x9e8\xdeq\x191\xa4\x8eQY\xb1\xc0,\x01 g\xbe\x0e=\r5\xa4\xba\xb8\x88\x86\x88 \x03<\x8c\xd4\xfar0\xb6PX\x1cz\n\x00\x9e\xf6\xd7\xed\xb6r[\x8b\x89\xed\xd9\xb0Vh\x1fk\xa3\x02\x08#\xb1\xc1\x03\x82\n\x9e\x84\x10H6(\xc5\x14\x01\xf0U\x85\xcf\xd8\xf5\x1bk\xad\xd3\xaf\x93*I\x9by|\xa9\x06\x08?#\xe0\xedn8l\x1c\x1epj{\x14\x92\xc3X\xb47Q\xc9\x0e\x1a9p\xeaA\xd8\xc02\xb6\x0fb\xa4\x10{\x82\rT\x10\xb3[\xbc\xe0\xc7\xb1\x1dP\x83"\x86\xcb\x02F\x179#\xe59 `q\x9cde\xf6\xb34N\xc8\xa22%\x01\x1btj\xc4\r\xc1\xbeRFT\xe5G#\x07\x19\x1d\t\x04\x03\xec]&{K\xad.\t\xcf\x9d\xba\xd1\x12\xed$\x8a\xd4\xc8\xc0`\x83\xb3(\xd9fP\xe8B|\xf8~1\xb9I\xecEq\xbe\r3\xb6\x85\xa5\xec\x95\x02,k\xe6\x07R\xc5\x97i\xe1NF\xd3\xb8\x83\x9ex\x04c\x9c\x8e\xc9zP\x075\xafZ=\xccW\x93\xcf\n\x06\x81\x8cv\xd2\x98\x947\x94\xc9\x19`\x1bq$\x17\x079\x0b\xf7G\xcap\x19\xbc?T\xb46\xcb!\x91&b\xd3\x90\x92\x96;Ov^G\'\xe6^\x87\x81\xc69\x18\xfa\x07R\x9a\xda\xebF7\x11N\xb2\xdbL\x81\xd2X\x88ud#!\x94\x83\x82\x08\xe4s\xcdx\x0e\xbe\xf1\xb6\xabt\x0c*U\xb7\r\x9b\x98\x05\xdc\x08\x04\x10s\x91\x9c\x8ez\x81\x9c\xd0\x06T\'|\xa9\x1c\x98\x8c\xb4L\xe5\xf9d\x05@\xf9r\x06rI\xc0\xe3\xeb\x81H\xeeLryc.\x9c\x00\xd9\x00\x9cg\xae:s\xd4f\xa6\xb7\xd4\xe5\x8a\x08b\x84\x98\xc8\x91gI\x16\r\xb2+`\xe3\xf7\x98\xdc1\x8e\x99\xc08\xe3$Q4\xb2\xcd#M!\x92i]\xf7;\x16\xcb1\'\x96$\x9ez\xe4\xd0\x07\xaa|$\x9d\xce\x992I:\xbb\x06\xe5\x00#\x1c\x0eq\xd8\x1f\xa9\xe9^\x88\xb1\xdcG\xe5\x85\xba\xf3\x14J\xef!\x9a0X\xa1\xdcB)]\xa0m%@$\x1f\x95prN\xea\xf0\x7f\tx\x95\xfc4.d(\xb2\x1b\x81\x98\x94v\xc1*w\x0c\xf1\xca\x93\xce8 \xf4 \x9dy~&kr\x07T\x8e\x04\x04c=H\xa0\x0fa\x91\xa1\xb9\x95\xad[\xcf\r\x1f\x971e\xdf\x18\xfb\xc4\xa8\xde0\x0f)\xca\xe4\xf0p\xc3\r\xcc7\xed\x04\xa9\x95{U\xba\x8bs\xdbI:y\x82\x19\n\x95\r\x8c\x83\xd1\x888 \x90H\xc8\xcdxS\xf8\xcb\xc4\x0e\xc7u\xfbc\xda\xb3\x9fR\xba\xba\x9d\xa4\xba\xbd\xb8\x04\x82r\x879l\x1c\x0cdpN\x01=\x878=(\x03\xdf\xa7\xd7\xec\xec\xa6\x95/\xae-\xe1\x01\xf6\xc7\x89r\xcc\xbbFK\x0c\r\xa7qa\x8ex\x00\xe7\x92\x06~\xa1\xe3\xcd\x0bOE-v\xb2\x93\xda2\x1b\xb6{W\x85O#\xbc\x10\xa3\xcc\xb2\x8f\x99\xfa|\xc8I\xc6\x0b\x11\x93\xc2\x82\x06H\x19\xecI\xa8\xa0\x8aI%\x10\xc3\x11\x92I\x81\x89QSs6\xe1\xb7\n?\xbd\xcf\x04r\x0e\x08\xc1\xc5\x00{\x05\xc7\xc5\x8d\x1c\xdb\xef\xb5\x8ei\x18\xa8*\xac\x85O\xe2\x0f"\xb2\x87\xc5Y\xaect\x86\xc1\x84\xb9\x01{\x8cs\x92N~\x9cc\xb9\xe9\x8e|\xc4-M\x04r4\x91-\xab\xcen\x9d\xcc~J)!\xb3\xb7n0~f\'#\x18\xe3\xb198\x00\xef\x97\xc6\x1e&h\x94=\xe5\xba\xbe\x06\xe2\x13\x03=\xf0\t8\xfc\xcdQ\x97Yid\x0fw\xe2\t\x84\xa4fDS\x80\x0eO\x03\x9eF1\xe9\\w\x9b+D\xd1\x99\xa41\xb3\x06d\xdepH\xce\t\x1e\xa3\'\xf34\xd8\x9eH&Ibc\x1c\xb1\xb0dt8e#\x90A\xech\x03\xb0mkB\x81\xf3p\xf3\xdf9\x1c31;9\xed\xfe{\xd3\xa4\xf1\x96\x99n\xa5\xac4\xa0d#\x1f\xbd\xe4t\xc6y\xef\\z\xa9\xb6\xb8\x8eH\xde)\n\xedpvn\\\xe0\x1c\x15a\x83\x83\xc1\x04\x10pz\x8al\x122\xccV\x1b\x94\x89\xf9\x8d\x9fy\x01w\x0c\x10\xc5A mnp\t\xc1\xe8h\x03\xa6\xbb\xf1\xad\xfa\x91\x14v6\xb13 o26V\x0b\x90\x08\x1c\x1e\xb8=;`\x83\x83\xc5f\x8f\x13\xeb\xf2\xab\x93w\x16\xd4R\xccr\x10\xe3p\x00\x00~\xf1\xe7\x9cs\xc6q\x8c\xe3%]\x96\xde;ub \x8c\x92\x91\x83\xf2\xa9 \x03\x81\xd0p\xaa?\x01\xe9MvX\xd5\x99\x88UQ\x92\xccp\x00\xf7\xa0\x0bKy}yp\xa8n\x9f|\x8c\x00-.\xc5\xc9\xf5$\x80\x07\xb9\xe2\xab2n%\x98\xe4\x9e\xa4\x9c\x9a\x94\xcb!\x80G!C\xc2\x02\xc2$V;Wh9\x00c#\xae:\x9eNM0\x93$j\x8f\x1cE"\x90\xbcm\xb4n\xc9P\t\'\x19\xf5\x18\xc9\x03\x9fS@\x05\xa8\x86;\x85i\xd7tk\x96)\x82C\x902\x14\xe1\x81\xc18\x04\x83\x90\t#\x9a\xad\xe4\x07H\xc4\xb8fB\x180\x04|\xc3\xb8\xe7\x8e\xfd\xfa\x1cU\xc5B~\xea\x96\xfa\x0c\xd4\xe9\xa7\xdd\xcc\xa8\xd0\xc2\xec]\x8a\xaa\x807dc\xb7P9\x1c\x9e\x0f8\xe8h\x02\x99\x8d3\xf2\xba\xba\xe0r\x14\xae\x0e9\x1c\x80x<g\xa1\xc7\x04\x8c\x12\x9bA>\xde\xd5\xb5i\xe0\xfda\x9c\x88\xf4\xf9T\xc8\xfb\x98\xb8\xeaOr\x7fO\xa0\x02\xba\x08\xfe\x18kM\x18s,\x08K`\xa6\x0e~\xbd1\xfa\xd0\x07\x0f\xe5~\xe9\xa5\xc2aX)\x05\x80\'9\xed\xd4\x8e\x0f#\x81\xc6z\x8aO)\x99F\xc8\x98\xfb\x05\xafW\xb6\xf8W\x1f\xd9!\x12N\x82p\xe1\x9d\xd9Kn\\\x8c\xae2\x07@p{\x16\xc9\xce1[\xb1x+N\xb3\x81m\x83L\x92NLq\xcb\x0c[\x8cm\xb4\x9d\xdc\xa9U\xc6\x0e\x0b\x0cg\x03\x92@ \x1eGig\xe2\x07ckga4M\t`\xea\xd1\x05m\xc7\x00\xe4\x91\xbb\x8d\xbd3\x81\x93\xc75\xadi\xe1_\x18\xdeM\x18yL1\x17\x01\xd9\x9c\x02\xa3\x07\x90\x07^\x9f\xe7\xbf\xb3[h\xb6\xb6\xd3<\xa8\xaf\xb9\xd5P\x82\xe4\xa8\x03$as\x80~c\x92\x06O\x19\xe81qm\xe3C\xc2\x8a\x00\xf3W\xf8q\xa8\xcf\x1b\xa7\xfc$\x17\x01\x8f\xdcfL\x8fl\xaezz\x80G\xb1\x1dkj\xcb\xc0\x1adr7\x9fl\xac\xb1I\x98\x18\xc8X\x91\xb4\x02\xc4\x106\x9c\x97\x18\x04\xf1\xcey u\x11G\x0e\xa3a<W\x12Z^\xc1#\xcd\x0c\x81\x10\x18\xd9C\xb2\x98\xd8\x12\xc0\x90\x06\xc6\xf5 \xf0:\x0b\x9bE\x00d\xda\xe8\xf6\xf0Hb[+q\x02\xa2\xec|\xe5\xcbd\xee\x04c\x80\x06\xdc\x1c\x9c\xe4\xf01\xcb\xe76p\xc1ym\xa9\xc1m\x1d\x8bE#3H\x0bB\xd0\x05]\xe6RT"r\xcc6\x92r\xa0\x9f\xef\x05\xd00\xa3\\\xa4\xe4\xbe\xf4F@\x04\x8c\x17\x04\x82r\xb9\xc1?(\xc1##\x9cc\'1\xddY\xc5y\xe4y\xaf:\xf92\xac\xcb\xe4\xce\xf1e\x87@\xdb\x08\xdc\xbc\xf2\xad\x95=\xc1\xa0\x06Z\xfe\xf3K\xb7"Y\xe5\xdd\n\x9f2\xe2?.G\xe0r\xeb\xb5v\xb1\xee6\x8c\x1e0:T\xd0G\xe5\xae=y\xa6[\xde\xdb\xdd\xcfw\x0c2o\x92\xd2Q\x0c\xe3i\x1b\x1c\xa2\xc8\x07=~WS\xc7\xaf\xaeh\x99\xee!\xfbD\xca\x9e|k\x101\xdb\xc4\xa0H\xce7\x1273\x059\xf9@\x07n\x0898<\x00I\x0c\x8f*\x16xd\x84\x87e\xda\xe5I 1\x01\xbeRF\x08\x19\x1d\xf0F@9\x02J\x8c4\xdfhud\x8c@\x11J8rX\xb6N\xe0W\x18\x00\r\xb899\xc9\xe0c\x99(\x03\xe0*tg\x12\xa1=7\ntr*$\xca\xd0\xc7!t\xda\xac\xc5\xb3\x19\xdc\x0e\xe5\xc1\x038\x04s\x91\x86<g\x04G@\x1fb\xf8\x01\x95\xfc?k*\xdc\x19U\xd1J\xa9\xc6\x10\x05\x03\x03\x00q\x90O$\x9c\x93\xdb\x00w\x0b\xd0W\x95|,\xbe\x0b\xe1\xdd&\xce8\x9efh\x8bJ\xd1\xbab\xddy\xda\xce\x0b\x06\xc3\x10\xc0m\x07\x959\xc63^\xa1\x05\xc434\xa9\x14\xb1\xbb\xc2\xfb%U`J6\xd0\xd8oC\xb5\x94\xe0\xf6 \xf7\xa0\n\xb7\x960-\x84\xb0\xc5\x1aC\x1b39\x11\xa0\x03s1fl\x0cd\x92I>\xa4\x9a\xf0/\x15\xda\x9bMu\xc6r$\x1b\xbe\x95\xf4\x1e\xa3\x1bKa<i+\xc2\xed\x1b*\xca\x80\x16BGQ\xb8\x11\x91\xd7\x90G\xb5xO\x8e\x97n\xb3\x13\x1f\xe2C\xfc\xe8\x03\x98Y\x9e\x16u\x17\x0c\x8b:\x88\xcc{\xb0\x1f\x07v=\xfa\x03\x8f\xf6A\xedR\xb1\x8c\xa4{\x15\x83\x05\xc3\x92\xd9\x0cry\x1cp1\x81\x8ey\x04\xe7\x9c\x08\xed\xa5@\xf1\xcd\xb5\'\x8f!\xf6\x92v\xb8\xeb\x8c\x82\x0e\x0f\xb1\x1fZf\x02J\x19\x9aBX\x90\xa3\x9c/\x03\xd3\xfd\xde\xa7\xd7\x19\xe7\x14\x01*nTU,X\x81\x82\xc4\x0c\x9f~*\xdcs\xdd\xcbll\xa3i\x9e\x00\xc6c\x02\x92W!N_oL\x85\x07\'\xd0TR\xca\x92\xe7e\xbcQe\xd9\xfeB\xc7\x00\xe3\x0b\xc9<\x0cq\xdf\x93\x92x\xc3\x0cO\x0b\x18\xa5h\xdaD;Y\xa3 \xa9#\x82A\x04\x82>\x84\xfdM\x00\x0f\x0bE!\x0e\xe1\xb7\x05q\x86V\n\n\x83\x8c\x8f\xd4\x1eA\xc88\xc6\x02\xb5\xb2\xac\x97\x12\xc0\xe8m\xf7\x80\x0b*\xa4\x8cNp\xc5w\x1c\xf0\xa3\xa6q\x803\xc8\xa87*\xb3N\xec#@\xa48v\xe9\x83\xc3u\xc0\x18\xc9\xf5\xe4g\x18\xc5O4k$ksm\x11\nCD\x92\xb8\xde7\x80\xbb\xc6@\x1f\xc5\x82W\xa8\xe0\x13\xde\x80\x06\x115\xb1\x8c\xed2\x17\r\x91\xbc2\x01\xd3\x9e\x14\x86\'\x91\xc9\xf9y\xc0?4i\x1a\xa2*"\x85U\x18\n\x06\x00\x1e\x95gi\x964)\x00]\xb8B\xd9#{\x16\xc0\xceN2I\n1\x8e\xdcg\xac\x90h\x97\xa8\xc4G`"\xde\xc5\x98\xe0($\xf5\'\xde\x80!\x8f\xcaX%\xdc\xbb\xa5m\xaa\x99S\x85\x19\xc9l\x829\xe0\x0c\x10A\x0cz\x10)\x8f\xe5\xecM\x8a\xc1\x82\xfc\xe4\xb6C\x1c\x9eG\x1c\x0c`c\x9eA9\xe7\x03\\\xf8~\xf1$(\xedo\xc1\xc1+&\xef\xcb\x1c\x1a\xb4<;p\xf1\xf9R\\L\xd1\xab\x97\x8d\x122\x02\xe4\x00{\xf5;G<p\x00\xed@\x1c\xe4\xb1\xb4\nL\xe8b\x01\x04\x84\xc86\xe1H\xdc\x1b\x9e\xd8 \xe7\xd2\x9c\xe22\xb1\x84VV\x0b\xf3\xee`w\x1c\x9e@\xc7\x03\x18\x18\xe7\x90Ny\xc0\xec\xac\xbc\x1e\xa6\x1b_\xb5\xda\xb2\xcf9\xd8\xaaf\xc0w\xdaX\xaa\xe4\x02\xdc+\x1e\x99\xc0\'\x15\xd5\xe9~\n\x82(Hm6=\xc3\xa1\x90\xe4\xe6\x80<~\x08\x1c\x03\x1cq\xeeU\xc0D\x8d\x0f\xca\xb8\x1c\x7f\x9cU\xd8t}B\xee\xf4[\xdbY\xbc\xf9T!\xd5J\xa8-\x8c\x86,\x00\\\x12\x06s\x8c\xe7\xb6\t\xf7+O\r\xc5o\x92\x91A\x11#\x9d\x8b\xcd_\x87LX\x98\x169?LP\x07\x89\xb7\x82\xf5\xfb\xadBg6\xdb\xa6\x92O\x9e\xe9\xf2\xdeh\xdcI98nrH\xcf\xaeH\xedZ\xb6\x7f\x0bo\xe5F\x9e+\x9f\xdd\\\x11"\x99`\xf2\x9f\x04\x0cnS\xca\x9c\x01\x90y\x1d;W\xb2,\x08\xa3\x85\xa6[^Cu\xb7\xcbY\xd71$\xc3\xcd\x81\xe3\xf9_8\x1f0\x18o\x94\xe5~\xf2\xf1\x9022\x01\xe76\x7f\x0c\xecn\xd5\xa7\xfbb\xb02:\x95\x81\xb7"2\xb1V@I\'\xe5 \xaf$\x90F\x0f5\xafg\xf0\xdfN\xb6HP\xed\x94+\xb1\x95\xa5BY\xd4\xee \x02\x08\nA*3\x83\xc2\xe0\x8c\x9d\xd5\xdb\xe4z\xd4p\xc9)BgH\xd1\xf7\xb0\x01\x1c\xb0\xdb\xb8\xed9 rW\x04\x8e\xc4\x91\x93\x8c\x90\x0c\xbb?\x0bi\x16.\xcd\r\x94C=\x88\xce?:\xbbomi(\xf3\xa3\x83n\xd6h\xf2\xf1\x14?)+\xc0`\x0e8\xe0\xf4#\x04d\x10j\xd1\x90\x0e\xf4\xdf5}h\x02#b\xaf\xe6\t\'\x9d\x95\xa5IUC\xec\xf2\xf6\xed!AL\x12\xa4\xaeHbs\xb9\x81\xf9N\xdaw\xdaDs\xf9w&\x08\xbc\xd9|\xbbQ\xe6\xe5\xa6\xf97\x1e\x08\x18n\x1f\xe5\x19\xf9W9\xea\x03\xbe\xd0\x83\xbd0\xdd\xc6?\x88P\x04\xb2\\C\x0b\xc2\x92\xcb\x1a<\xcf\xb2%f\x00\xbbm-\x85\x1d\xce\xd5c\x81\xd8\x13\xda\xa3S-\xc7\x97!\xf3\xed|\xb9_tga\xf3TnQ\x9cn\xf9O\x0e0Cp\xb9\xc7\xcc\xb5\x13_\xc6?\x88Ux\xb5L\xdbD\xd7\x06(\xe7(\x0c\x88\x8e]U\xb1\xc8\x0c@\xc8\xcfC\x81\xf4\x14\x01vS\x15\xb4\xadt\xdeyi<\xb8HM\xf2\x01\xf3\x10\xa4 \xc8\x1c\xbf-\x81\xc0\xcb\x1c/\x0f\x95\x19\xde\x16Y\xde0\x8f\xb9\x95B\xe2A\xb4\x8d\xad\x90N2A\xe3\x07*9\xc6A\xcc\x97Y\x82 KH\xa0c9\xdc1YPx\x96\xce\xd6\xc8\x19\xee!G9\x92Q\xf6\xa3(Fc\xb9\x80f\xc1*\t p\x00\x18\x00\x00\x00\x00\x1dCE\x03\\%\xc3E\x19\x9d\x11\x91$*7*\xb1\x05\x80=@%W#\xbe\xd1\xe9QL\xd6\xd2^[\xa4\xb0\xef\x997M\x0b\xb4%\x84d\r\x84\x86\xc6\x15\xb1!\x03\x90H-\x8c\x80\xd5\xe7\xda\xa7\x8ft\xa9\xe7\xb0{]r\xda/\xb3\xdd\t%\xfd\xf1\n\xe9\xb1\xd4\xa9Q\xf7\xfe\xf7\x00\x90\x03\x05nv\xedj\x97_\x18\xbc5\x04\xcd\x13j\x00\xb2\xf0J\xa9"\x80=:\xe1\xe46\xf2\xad\xbc\x91\xc79B#y\x10\xba\xabc\x82T\x10H\xcfl\x8c\xfa\x8a\x86\xf5V\xea\x05\x8f\x106%\x8eLO\x17\x98\xbf#\xabp20\xdc|\xa7\xf8[\x07\x07\x18>3?\xc7\r\x02\xd1\xda;U\xbc\x9e&f\x90331\xf9\x89b2\xdc\xf7\xe0t\x03\x80\x00\x00Ve\xff\x00\xc7\xbb`\xaal\xac%s\xdc9\xc5\x00{\xdc7\x0e\x96\xf1\xad\xc4\xb1\xc98@$x\xd0\xa2\xb3c\x92\x14\x92@\xcfl\x9cz\x9ak\xdd\xb0\xb8B%\x8c@\x11\x83\xa1BX\xb6F\xd2\x1b8\x00\r\xd9\x189\xc8\xe4c\x9f\x9dn~._K\xa1\\\xea0hW\x10$\x97\x91\xaf\xda\xa3FX\xa4\x94(8yT\x8c\xb8T\x8cm\xe7r\xe4\x1e\x06\x0f8\xbf\x18\xbcF\x91yHS`\x8dcP\xe4\xb9\\g\x9d\xc7\xe6$\xf1\x92I\xe9\xf5\xc8\x07\xd5\xa6\xfe!\x9f\x9cW?\xfd\x9f\x7f\xff\x00C\x9e\xb7\xff\x00~\xac\xbf\xf9\x1e\xbea?\x14<XN\x7f\xb4\x8f\xfd\xf2)?\xe1g\xf8\xb3\xfe\x82g\xfe\xf9\x14\x01\xc9\xc3\x1a\xca\xe5^h\xe1\x01\x19\xb78b\t\nH_\x94\x13\x92F\x07l\x91\x92\x06H&\x99\xa7p\xee#\x04"\xa7\xc9\x1a\xa0\xc2\xa8Q\xc2\x803\x81\xc9\xeaNI\xc9$\xd0 \x99\xad\xde\xe1b\x90\xc0\x8e\xa8\xf2\x05;U\x98\x12\xa0\x9e\x80\x90\xac@\xef\xb4\xfaU\xcc#\xeb\x1eN\x8b\x04\xf7\x0b/\xfa<\t<+$\x92\x96]\x84\x84\x00\x80\xccI*\x06J\x920\xc4\x80\xc4\x03\xe9\xef\x84\xce\x1b\xc2\x16\x00\x10~Lu\xafMO\xba+\xc7~\n\xddIs\xe1\xb6\x96O,4\x93\xb3\x90\x91\xaa($\x93\x85U\x00(\xe7\xa0\x00\x0f\xc2\xbd\x85\x0f\xca(\x01\xb7912\x8e\xe0\xd7\x89\xf8\xfe\xceO>\x19\x80\xce\xc2T\xe0d\xd7\xae\xcbo\x1d\xdc\x13=\x9c\xd3X<\xb2\xc8e\x92(\x15\x1eG\nb\xdc\xc2D9\xc6\xd5*\xd8\xe7brS\x83\x8fq\xa0=\xd4\xf2;\xca\xa03d\x02\xb9\xa0\x0f\x10\xb7\xb2\xbe\xb9\xb5\x91m\xed\x9d\xd0:\x97o+88l\r\xd8\xc8\xcf\xcd\xc6y\xc7\xb0\xc4\x96^\x1c\xbf\xbc\xbc\x16\xe5\xddd\xb7\x00\xba\xbf\xcb\x9d\xc3\x86#\x03 \xe1\xb0G\x19\x07\xba\xf1\xec\xb6\x9e\x161\xb8k\xadFk\x8cD\x91\xfc\xc1T\xfc\xa3\x1b\xbeP9#\x1ct\xe0`\n\xbbm\xe1\xfd:\xd1#@\xd2\xba\xa2\x85\x1ed\xa5\xc9\x03\xd4\x93\x92}\xcf4\x01\xe5?\xf0\x85^.\xd63#\x8d\xca\n\xc7\x8c\x80H\x04\xf2G\x00\x1c\x9e\xf8\x1cd\xf1Z\xdf\xf0\x81X\xba\x80\xf7\x13\x9eA\xf9r:s\xda\xbd)\xec4\xbd\xf0\xbf\x92\x01\x89\xcb\xae\xd6+\x93\x82>l\x1f\x98`\xf49\x19\xc1\xea\x01\xa9\xa6{\x19\xa3\x11\xcd\x0c2"\xb2\xb8W@@e!\x94\xf3\xdc\x10\x08=\x88\x06\x8088|\x19d\x90\x85\x8fMiO\xf7\xdcu\xab0\xf82\xe8\x17\xf2!H\x93\x00\x08\x9dW\nFrG\x19\xc9\xc8\xeaq\xc0\xc69\xcfc\x0e\xaf\x1b\xdb\xc7#\xa1\xb7f@\xc6)\n\x96BG\xdd;I\x19\x1d8${\xd43x\x82\xd6\x13\xf3\xcd\x1a\x1e\xd9`(\x03\x9d\x8b\xc0\xb7S\xabGwr\x8b\t!\xbc\xb5@\xcaXg\r\x821\x91\x9c\x83\xd4\x1eF\rZ\xd3\xfc\'cfQnnf\x89\xae%hR\x19\xe5\\\xcd\xb4\x96\x18\x00\x9c\xe4!p:\xe0d\x80A\x02\xcd\xdf\x8c\xb4\xcbH\xfc\xc9\xef`D\xf5.+2\x7f\x89>\x1f\x8a=\xff\x00\xda\xd6\xcc\x00\xc9\n\xfc\x8a\x00\xe9\xe3\xf0\xfe\x97\t\x05m\xd3#\xda\xaf\x08-\x93\xa4H?\n\xf36\xf8\xc3\xe1\xaf\'\xcc:\x94aX\xba\x8c\xf2\xd9P\x0f\xdd\x1c\x8c\xe4\x00H\xc19\xe7\x83\x8en\xe7\xe3\x9e\x85$\xa8Z\xcaiZ\x197\xc5# %\x1b\x05r\xbe\x84\x86#>\x84\x8e\xe6\x80=\xc2B\x0bBRg\x89c}\xcc\xa8\x17\x12\r\xa4ml\x82q\x92\x0f\x189Q\xce2\r%\xbc\xb1\xd3\xcc\xcds\xa9o\x9a\xde\xd63s$\xf3\x81\x88\xd7~%e\x18D\xc9\x0f\x96\n\xa0\xed\xf4P\x07\x83\xdc|z\xf2\xeee\xf2,g\xb8\x81\xce\xe5Y$T\xf2\xfeP00\xbd27s\x92rF@\xc6)\xea\xdf\x1b/\xa0\xd5V+E\xb6\xbb\xb4F]\xf2[\xb3\xa8\x94pHS"\x02\xa7\xa8\xc9^\xa3\xa1\x1d@>\x89\xb9\xd5\r\xbc\xc8\x8bi4\xc1\xbe\xf3!@\x13\xe6U\xe7s\x03\xd1\x8bp\x0f\x08\xdd\xf6\x86\x8e\xe3W\xf2\x0bI\xb05\xbcq<\x92\x14\xdc\xf2ep@TU%\xb27\x1e\x0ex\x00\x03\x9e>W\x9f\xe2\xf7\x88\xde\xd4\xc1l-\xed\x14\xbbI\xfb\x94\xc7\xcc\xc7,q\xeaI$\x9e\xe6\xb3e\xf8\x97\xe2\xb9cdmM\x80a\x8c\x85\x19\xa0\x0f\xae\x9fW\x85\x7f\x8cU7\xf1-\x92\x12\r\xcc@\x8f\xf6\xc5||\xde"\xd7\xc5\xb2\xce\xda\xbc\xe5$vP>\xd0\x0bd`\x9c\xaer\x07\xcc0H\xc1\xe4\x0e\x87\x19\x8d{t\xecY\xaee,NI.h\x03\xec+\xaf\x1fh\x96\x93ys\xea\x16\xe8\xdd\xd4\xc9\x8a\xe5\xf5O\x89>\x1d\xb1:u\xbb^-\xcd\x9a\x0c8\x92Q*\x82\x85Z7%\x83;H\x19A\x07 r\xccN\xe0\xb9\xf9\x81\xdd\xe4m\xce\xcc\xc7\xd4\x9c\xd5\xacB^\xda\xeekh\xd6\xd0\xba\xc6\xf0\xdb\xdc\x05v\xd8\xa9\xbc\xfc\xc5\x8a\x16\xcew\x11\xb7%\xb6\x8c)P\x01\xf4<\xdf\x19\xfc3\xa7YG\x15\x93f(\x94G\x1c\x10G\xb4"\x81\x80\x00\xe8\x00\x1d\xab%\xbe:iQ\xf9\xef\r\xb5\xe3<\x8d\xbc\x87rFp\x07\x00\x9f\x94p8\x18\x19\xc9\xeaI>\x14\x97\x8f\x15\x9b[\xc4<\xaf3"gG`fBP\x84a\x9d\xa5C a\xc7S\xc98\\W\xa0\x0fg\x93\xe3\xe5\xc3\x9f\xf9\x05{}\xfa\xc8\x9b\xe3&\xbfus-\xcc\x10\xdb\xc6\x90FN\xc7\x9bnU\x99G\nH\x0e\xd9\xc7@H\x1b\x8f\x03uy}X\xb1\xb5\xfbu\xe4v\xc2\xe2\x08\x1aL\x85y\xdfbn\xc1\xc0-\xd1rp2p\xa39$\x0c\x90\x01\xd8^|X\xf1U\xcc\xc5\xd2\xf1a_\xee"\xf1P]x\xbf\xc5\x1a\xc1\x8cA\xaa\\\xb4\x1b\xe2\x89\xa7p\xb0"J\xeb\xf7Y\xcbmQ\x95|\x16#!K\x109\x03\x92\x86y\xad\x9c\xbc\x12\xc9\x13\x94d,\x8cT\x95e*\xc3\x8e\xc5I\x04w\x04\x8a\x8e\x806\xe4\xf1>\xad\xe5 ]V\xeac$L\xb3$\xe8\n\xa9m\xcb\x85\xc99\xf9H!\xb0\x08c\xc7\xdd\x0crZ\xe6v\x044\xd2\x10x \xb9\xa8\xa8\xa0\x02\xacY\xb6e6\xe6X!\x8e\xe3lRM4{\xc4k\xb8\x1d\xdc+2\xe3\x03%F\xecdr\t\x062!\xfb:2\xc9!\x9c\xbb\x07B\x80(\\\r\xa46rI;\xb201\x81\xc9\xcf\x11\xd0\x04\x93F\xb18T\x9a9\x81Em\xc8\x18\x00J\x82W\xe6\x00\xe4\x13\x83\xdb \xe0\x91\x82c\xa2\xa40\xb2\xdb\xa4\xe4\xc7\xb1\xdd\x90\x01"\x96\xca\x80NW9\x03\xe6\x18$`\xf3\x8c\xe0\xe0\x02\xc5\xacv2"\xc74\xd2G<\xaf\xb3\xccq\xb6(\x06\xe4\xc4\x8cT38\xc7\x98\n\x85\x04aH-\xca\xd5:(\xa0\t#\x10\x94\x98\xcb$\x8a\xe13\x10T\x0c\x19\xb7\x0e\x18\xe4m\x1bw\x1c\x8c\xf2\x00\xc79\x11\xd4\x90\x08Z\xe2%\xb8\x92H\xe0.\x04\x8f\x1a\x07e\\\xf2B\x92\x018\xed\x91\x9fQQ\xd0\x01E\x14P\x07\xd1\xbf\x05nb\xb5\xf0\x8aI4\xb1\xc6\x86m\x81\xa4`\x06\xe6!Ts\xdc\x92\x00\xf7"\xbd,x\x8bA\xb4\x92}\xb7v\xd1\xcb#\xee\x98\xa8\xc1v\n\x17-\xc7\'\n\xa3\'\xb0\x03\xb5\x14P\x075s\xf1s\xc2\xb0\\\xc9\x05\xc6\xa0`\x926u*\xeb\x9c\xe0\x9c\x11\xb7?{\x00\x8f\xa8\xce9\xc71\xaa|i\xd0-o\xda[\x15Y\x1eD\x89Z\xe1c\xdc\xcc\xaa\xccJ\x11\xc7\x00\x13\xb4\xe4\xe0\xb98\xe3\xe6(\xa0\x0c\xedO\xe3\xdd\x9f\xd8\xdf\xfb:\xceSq\x82\x14\xca>\\\xf6$dg\xe9\x91X\xb7_\x1e/d\x85\x84\x1aj\xa4\x9d\x8b>@\xa2\x8a\x00\xc6\x7f\x8d>%y\x03b\x00\x02\x91\xb0/\x04\x9cs\xeb\x91\x8f^\xe7\xdb\x19\x97?\x15|Uq1t\xbcXT\x806F\xa7\x1c\x00\t\xe4\x93\x93\x8c\x9e\xd9<\x008\xa2\x8a\x00\xc9\xbb\xf1\xaf\x88\xaff2\xcb\xaa\xdc\x06\xf4F\xc0\xaa7\xda\xcd\xdd\xf2\xc0d\xba\xbbi\x15\x08\x91\xa4\x9fpf\xdcyP\x00\xda6\xed\x189\xe4\x13\x9ep\n(\x02\x1b\xe9\xb5\x15\x93\xecw\xf2]\x07\xb5-\x0f\x91;6a\xc3\x12Si\xfb\xbf1bG\xa94\xb76mco\x1a\xdd\xdb\xc8\x97\x13\xa4s\xc2|\xd5\xe2"\x1b\xef&2\x0b|\x8c\xb9#\xe5\xe7\x04:\x90Q@\x15\xc4jm\xdeS4a\xd5\xd5DD6\xe6\x04\x1c\xb0\xe3\x18\x18\x00\xe4\x83\xf3\x0c\x03\xce#\xa2\x8a\x00\xb0\x9fl\xd3\xe5\xb5\xbc\x8f\xcf\xb6\x90\xe2{i\x97(N\xd6 :7\xb3)\x19\x1d\n\x9e\xe2\xab\xd1E\x00X\xb7\x17\x06\x0b\xb3\x04\x1edb go$?\x96\x9b\xd7\r\x92\x0e\xcf\x9bj\xee\x18\xfb\xdbs\x86 \x9f\xba\xb5\x9f\xfeX^)\x8b\xfd\xb0\xaa\xcc\x9f\xf0\x13\xb9\t\xff\x00wr\xff\x00\x12\xf5(\xa0\x08\xc4,\xd6\xef81\xecGT \xc8\xa1\xb2\xc0\x91\x85\xceH\xf9NH\x18\x1cg\x19\x19\x8e\x8a(\x00\xa9&\x9ek\x97\x0f<\xb2J\xe1\x15\x03;\x16!UB\xa8\xe7\xb0P\x00\x1d\x80\x02\x8a(\x02O\xb4$pypE\x86\x92/.v\x94,\x9b\x8e\xfd\xc0\xa6W1\xf0\x14pI\xe1\xb9\xc3\x15\xaa\xf4Q@\x12G"\xa2L\xad\x0cr\x17M\xaa\xcc[1\x9d\xc0\xee\\\x103\x80G9\x18c\xc6pD\x96v\xdfh\x94\xb4\x8b8\xb5\x8bk\xdc\xcb\x0c^a\x8a2\xc1Kc u`\x06H\x04\x9023E\x14\x01$"\xd0Y\x11<\x91\x87\x91\xd8\x82\x88\xed,{P\xed\x18%Sc\xb3\x00O,6\x13\x81\xd1\xe9\xd1E\x00X\xb3m\xb3\xb1\xf3`\x8b\xf7R\r\xd3\xc7\xbdNQ\x86\x00\xda\xdf1\xe8\xa7\x1c1\x07+\x8d\xc29 \x9a\x14\x85\xe5\x8aDI\x93|L\xca@u\xdcW+\xea7+\x0c\x8e\xe0\x8e\xd4Q@\x04\x934\xa9\n0\x8c\x08\x93b\xed\x8dT\x91\xb8\xb7\xcc@\xcb\x1c\xb1\xe4\xe4\xe3\x03\xa0\x00\x00C\xf6wf\x92A8u\x08\x81\x01R\xb8;\x89l\xe4\x10v\xe0`\xe7\'\x91\x8eJ(\x02:\x923\x08I\x84\xb1\xc8\xceS\x11\x15p\xa1[p\xe5\x86\x0e\xe1\xb7p\xc0\xc7$\x1c\xf1\x82Q@\x11\xd5\x8bk7\xbb\xdb\x1d\xb9\xf3.\x9eT\x8a+TFi%-\x9f\xba\x00\xc1\xc1\x00c9%\x86\x01\xe7\x05\x14\x01\x1c\xd3\xcdr\xe1\xe7\x96I\\"\xa0gb\xc4*\xa8U\x1c\xf6\n\x00\x03\xb0\x00TtQ@\x1f\xff\xd9\ne\n\x0eimage/filename\x12S\nQ\nOScreenshot-2023-11-18-at-8-17-22-AM_png.rf.f19862e7e30f55f58ac30d96dc72373f.jpg\n\x18\n\x0cimage/format\x12\x08\n\x06\n\x04jpeg\n\x16\n\x0cimage/height\x12\x06\x1a\x04\n\x02\xb8\x02\n"\n\x16image/object/bbox/xmax\x12\x08\x12\x06\n\x04\x00\x00\x80?\n"\n\x16image/object/bbox/xmin\x12\x08\x12\x06\n\x04\x00\x00\x00\x00\n"\n\x16image/object/bbox/ymax\x12\x08\x12\x06\n\x04\x00\x00\x80?\n"\n\x16image/object/bbox/ymin\x12\x08\x12\x06\n\x04\x00\x00\x00\x00\n!\n\x18image/object/class/label\x12\x05\x1a\x03\n\x01\x05\n%\n\x17image/object/class/text\x12\n\n\x08\n\x06Square\n\x15\n\x0bimage/width\x12\x06\x1a\x04\n\x02\xb9\x02'>
)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




<tf.Tensor: shape=(), dtype=string, numpy=b'\n\x84\xfe\x01\n\xf6\xfa\x01\n\rimage/encoded\x12\xe3\xfa\x01\n\xdf\xfa\x01\n\xdb\xfa\x01\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x018\x019\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x9

In [ ]:
x = train_ds.map(lambda x, y: x)